In [1]:
import sys, psycopg2
import pandas as pd
sys.path.append('C:\\Projects\\uni\\sources\\companybot\\')
from datetime import datetime
import ujson, copy, json
from bots.config import DB_NAME, DB_HOST, DB_USER, DB_PASSWORD, CRUNCHBASE_DIR
from bots.common import get_data_diff, get_uuids_from_crunchbase_organizations, get_data_from_pending, DataSource, PendingStatus, get_persons, match_names, get_unique_officers, get_unique_shareholders, get_unique_founders
from bots.companieshouse_bot import CompaniesHouseBot
from bots.crunchbase_bot import run_crunchbase_bot
from bots.linkedin_bot import run_linkedin_bot

## experiment

In [2]:
import pandas as pd

# Sample DataFrame with additional data
df = pd.DataFrame({
    'id': ['SH01', 'SH01', 'SH01', 'SH01', 'SH01', 'SH01', 'SH01', 'SH02', 'SH02', 'SH02', 'SH02', 'SH02', 'SH03', 'SH03', 'SH03', 'SH03', 'SH03', 'SH03', 'SH03', 'SH03'],
    'name': ['John Smith', 'John Smith', 'John Smith', 'John Smith', 'John Smith', 'John Smith', 'John Smith', 'Jane Doe', 'Jane Doe', 'Jane Doe', 'Jane Doe', 'Jane Doe', 'Alice Brown', 'Alice Brown', 'Alice Brown', 'Alice Brown', 'Alice Brown', 'Alice Brown', 'Alice Brown', 'Alice Brown'],
    'parent_id': ['Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises'],
    'parent_name': ['Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises', 'Doe Enterprises'],
    'group_id': [1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1],
    'variable': ['is_passive_investor_', 'is_active_investor_', 'is_director_', 'shares_held', 'share_type', 'shares_held', 'share_type', 'is_active_investor_', 'is_passive_investor_', 'is_director_', 'shares_held', 'share_type', 'is_passive_investor', 'is_active_investor_', 'is_director_', 'shares_held', 'share_type', 'shares_held', 'share_type', 'is_passive_investor_'],
    'from_date': ['2020-01-01', '2020-06-01', '2020-06-01', '2020-01-01', '2020-01-01', '2020-01-01', '2020-01-01', '2020-01-01', '2020-06-01', '2020-06-01', '2020-01-01', '2020-01-01', '2020-01-01', '2021-01-01', '2021-01-01', '2020-01-01', '2020-01-01', '2020-01-01', '2020-01-01', '2021-01-01'],
    'to_date': ['2020-05-31', '2021-01-31', '2021-01-31', '2020-12-31', '2020-12-31', '2020-12-31', '2020-12-31', '2020-05-31', '2021-01-31', '2021-01-31', '2020-12-31', '2020-12-31', '2020-12-31', '2022-12-31', '2022-12-31', '2020-12-31', '2020-12-31', '2020-12-31', '2020-12-31', '2022-12-31'],
    'value': ['true', 'true', 'true', '1000', 'TypeA', '2000', 'TypeB', 'true', 'true', 'true', '5000', 'TypeA', 'true', 'true', 'true', '2000', 'TypeA', '3000', 'TypeB', 'true'],

})

# # Pivoting the preprocessed DataFrame to wide format and unstacking by common_id
wide_df = df.pivot_table(index=['id', 'name', 'parent_id', 'parent_name', 'from_date', 'to_date', 'group_id'],
                         columns='variable',
                         values='value',
                         aggfunc='first')
df.columns

Index(['id', 'name', 'parent_id', 'parent_name', 'group_id', 'variable',
       'from_date', 'to_date', 'value'],
      dtype='object')

In [72]:
companieshouse_sample = """{"properties": {"parsing_date": "2023-08-11", "company_name": "Isize Limited", "uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "company_id": "10388803", "url": "https://find-and-update.company-information.service.gov.uk/company/10388803", "key": "company", "address": "3rd Floor 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "status": "Active", "type": "Private limited Company", "incorporated_on": "2016-09-22", "sics": ["59120 - Motion picture, video and television programme post-production activities", "62090 - Other information technology service activities"]}, "cards": {"officer": {"url": "https://find-and-update.company-information.service.gov.uk/company/10388803/officers", "items": [{"name": "Charles Ronald Conn Iii", "appointments_url": "https://find-and-update.company-information.service.gov.uk/officers/_kCUD6uPdwLMYcg1ay2qstFJuFI/appointments", "address": "9th Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "status": null, "role": "Director", "appointed_on": "2020-11-12", "resigned_on": "2022-06-29", "date_of_birth": "1961-08-01", "nationality": "American,Canadian", "residence": "England", "occupation": "Company Director"}, {"name": "Edward Clifton", "appointments_url": "https://find-and-update.company-information.service.gov.uk/officers/xjirzm9boUZhyX_quiHEhWsLrl0/appointments", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "status": "Active", "role": "Director", "appointed_on": "2022-07-20", "resigned_on": null, "date_of_birth": "1971-01-01", "nationality": "British", "residence": "United Kingdom", "occupation": "Director"}, {"name": "Hugo Marcelo Muriel Arriaran", "appointments_url": "https://find-and-update.company-information.service.gov.uk/officers/GdP1ZTjt4-Za4T1MsRTb6dYSlFY/appointments", "address": "3 Falconet Court, 123 Wapping High Street, London, United Kingdom, E1W 3NX", "status": null, "role": "Director", "appointed_on": "2017-02-03", "resigned_on": "2018-11-05", "date_of_birth": "1983-10-01", "nationality": "Swiss", "residence": "Switzerland", "occupation": "Computer Scientist"}, {"name": "Ioannis Andreopoulos", "appointments_url": "https://find-and-update.company-information.service.gov.uk/officers/7dK_qGQnZEYGIus3DO_O-QANpho/appointments", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "status": "Active", "role": "Director", "appointed_on": "2018-11-05", "resigned_on": null, "date_of_birth": "1977-01-01", "nationality": "British", "residence": "England", "occupation": "Chief Technical Officer"}, {"name": "Marko Milos", "appointments_url": "https://find-and-update.company-information.service.gov.uk/officers/3oUbsQ1L6sfBFt4cR00eiL38o8M/appointments", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "status": "Active", "role": "Director", "appointed_on": "2017-03-27", "resigned_on": null, "date_of_birth": "1978-04-01", "nationality": "English", "residence": "England", "occupation": "Banker"}, {"name": "Oakwood Corporate Secretary Limited", "appointments_url": "https://find-and-update.company-information.service.gov.uk/officers/qkCSxkXy04LQ3YY6ZfkveKpkaCM/appointments", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "status": "Active", "role": "Secretary", "appointed_on": "2023-07-12", "resigned_on": null, "date_of_birth": null, "nationality": null, "residence": null, "occupation": null}, {"name": "Ohs Secretaries Limited", "appointments_url": "https://find-and-update.company-information.service.gov.uk/officers/8DTKf650Reo5ubQ17kwO3hBSxt4/appointments", "address": "9th Floor, 107 Cheapside, London, England, England, EC2V 6DN", "status": null, "role": "Secretary", "appointed_on": "2021-01-31", "resigned_on": "2023-07-04", "date_of_birth": null, "nationality": null, "residence": null, "occupation": null}, {"name": "Simon Paul", "appointments_url": "https://find-and-update.company-information.service.gov.uk/officers/K1hjAdUc1lmw5-PCEBzC78AdUeQ/appointments", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "status": "Active", "role": "Director", "appointed_on": "2021-04-01", "resigned_on": null, "date_of_birth": "1984-05-01", "nationality": "British", "residence": "United Kingdom", "occupation": "Director"}, {"name": "Srdjan Grce", "appointments_url": "https://find-and-update.company-information.service.gov.uk/officers/5jfbbnRTFU8OF_-nOcvN2wcMReo/appointments", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "status": "Active", "role": "Director", "appointed_on": "2016-09-22", "resigned_on": null, "date_of_birth": "1977-10-01", "nationality": "British", "residence": "United Kingdom", "occupation": "Founder & Ceo"}]}, "insolvency": {"url": "https://find-and-update.company-information.service.gov.uk/company/10388803/insolvency", "items": []}, "shareholding": {"2022-09-21": {"source": "Confirmation Statement", "url": "https://find-and-update.company-information.service.gov.uk/company/10388803/filing-history/MzM1MjUxNTYzNmFkaXF6a2N4/document?format=pdf&download=0", "received_date": "2022-09-21", "filing_date": "2022-09-21", "update": true, "items": [{"name": "TD VEEN AS", "share_type": "DEFERRED", "shares": 32486, "is_company": true}, {"name": "A MANAGEMENT AS", "share_type": "ORDINARY", "shares": 3898, "is_company": true}, {"name": "HASAN ALAKAN", "share_type": "ORDINARY", "shares": 4873, "is_company": false}, {"name": "AMADEUS CAPITAL PARTNERS (NOMINEE) LTD", "share_type": "ORDINARY", "shares": 9746, "is_company": true}, {"name": "IOANNIS ANDREOPOULOS", "share_type": "ORDINARY", "shares": 484371, "is_company": false}, {"name": "ADRIANA BELONIC", "share_type": "ORDINARY", "shares": 2500, "is_company": false}, {"name": "ALEKSANDRA BELONIC", "share_type": "ORDINARY", "shares": 5000, "is_company": false}, {"name": "MEREDITH BENNETT", "share_type": "ORDINARY", "shares": 1949, "is_company": false}, {"name": "ANDREA BUCCA", "share_type": "ORDINARY", "shares": 4873, "is_company": false}, {"name": "CHARLES RONALD CONN III", "share_type": "ORDINARY", "shares": 7797, "is_company": false}, {"name": "VOJISLAV DJURIC", "share_type": "ORDINARY", "shares": 40000, "is_company": false}, {"name": "SRDJAN GRCE", "share_type": "ORDINARY", "shares": 484371, "is_company": false}, {"name": "GYLDENLAKK AS", "share_type": "ORDINARY", "shares": 2923, "is_company": true}, {"name": "ISIZE CORPORATE TRUSTEE LIMITED", "share_type": "ORDINARY", "shares": 158000, "is_company": true}, {"name": "SANIA JEVTIC", "share_type": "ORDINARY", "shares": 5888, "is_company": false}, {"name": "ALEXANDER KOLB", "share_type": "ORDINARY", "shares": 14699, "is_company": false}, {"name": "ANDREW LOFTUS", "share_type": "ORDINARY", "shares": 7837, "is_company": false}, {"name": "STEVEN MARSHALL", "share_type": "ORDINARY", "shares": 17623, "is_company": false}, {"name": "PAUL MASSARA", "share_type": "ORDINARY", "shares": 4873, "is_company": false}, {"name": "DEJAN MILADINOVIC", "share_type": "ORDINARY", "shares": 16500, "is_company": false}, {"name": "MARKO MILOS", "share_type": "ORDINARY", "shares": 38000, "is_company": false}, {"name": "HUGO MARCELO MURIEL ARRIARAN", "share_type": "ORDINARY", "shares": 3858, "is_company": false}, {"name": "OHANA AS", "share_type": "ORDINARY", "shares": 2923, "is_company": true}, {"name": "DHRUV PANDYA", "share_type": "ORDINARY", "shares": 5888, "is_company": false}, {"name": "DUSAN PETROVIC", "share_type": "ORDINARY", "shares": 73000, "is_company": false}, {"name": "PATRICK PICHETTE", "share_type": "ORDINARY", "shares": 4873, "is_company": false}, {"name": "RADKA INVEST AS", "share_type": "ORDINARY", "shares": 1949, "is_company": true}, {"name": "ROBERT SANSOM", "share_type": "ORDINARY", "shares": 7797, "is_company": false}, {"name": "PAOLO SIDERI", "share_type": "ORDINARY", "shares": 974, "is_company": false}, {"name": "MARIA SILVESTRIS", "share_type": "ORDINARY", "shares": 5888, "is_company": false}, {"name": "TAMORER PTY LTD", "share_type": "ORDINARY", "shares": 29238, "is_company": true}, {"name": "TD VEEN AS", "share_type": "ORDINARY", "shares": 38010, "is_company": true}, {"name": "WILLIAM TUNSTALL-PEDOE", "share_type": "ORDINARY", "shares": 3898, "is_company": false}, {"name": "CHRISTOPHER WADE", "share_type": "ORDINARY", "shares": 15000, "is_company": false}, {"name": "TINA WHITEHOUSE", "share_type": "ORDINARY", "shares": 7837, "is_company": false}, {"name": "COLIN WOODFORD", "share_type": "ORDINARY", "shares": 6862, "is_company": false}, {"name": "A MANAGEMENT AS", "share_type": "SERIES SEED 1 ORDINARY", "shares": 2046, "is_company": true}, {"name": "GYLDENLAKK AS", "share_type": "SERIES SEED 1 ORDINARY", "shares": 1534, "is_company": true}, {"name": "OCTOPUS TITAN VCT PLC", "share_type": "SERIES SEED 1 ORDINARY", "shares": 335249, "is_company": true}, {"name": "OHANA AS", "share_type": "SERIES SEED 1 ORDINARY", "shares": 1534, "is_company": true}, {"name": "RADKA INVEST AS", "share_type": "SERIES SEED 1 ORDINARY", "shares": 1023, "is_company": true}, {"name": "TAMORER PTY LTD", "share_type": "SERIES SEED 1 ORDINARY", "shares": 15346, "is_company": true}, {"name": "TD VEEN AS", "share_type": "SERIES SEED 1 ORDINARY", "shares": 147496, "is_company": true}, {"name": "CHARLES RONALD CONN III", "share_type": "SERIES SEED 2 ORDINARY", "shares": 4092, "is_company": false}, {"name": "CHRISTOPHER JOHN MAIRS", "share_type": "SERIES SEED 2 ORDINARY", "shares": 2554, "is_company": false}, {"name": "OCTOPUS INVESTMENTS NOMINEES LIMITED", "share_type": "SERIES SEED 2 ORDINARY", "shares": 111750, "is_company": true}, {"name": "PATRICK PICHETTE", "share_type": "SERIES SEED 2 ORDINARY", "shares": 2557, "is_company": false}, {"name": "ROBERT SANSOM", "share_type": "SERIES SEED 2 ORDINARY", "shares": 9207, "is_company": false}, {"name": "WILLIAM TUNSTALL-PEDOE", "share_type": "SERIES SEED 2 ORDINARY", "shares": 2046, "is_company": false}]}, "2021-09-24": {"source": "Confirmation Statement", "url": "https://find-and-update.company-information.service.gov.uk/company/10388803/filing-history/MzMxNDg2MjY0M2FkaXF6a2N4/document?format=pdf&download=0", "received_date": "2021-09-24", "filing_date": "2021-09-21", "update": true, "items": [{"name": "TD VEEN AS", "share_type": "DEFERRED", "shares": 32486, "is_company": true}, {"name": "A MANAGEMENT AS", "share_type": "ORDINARY", "shares": 3898, "is_company": true}, {"name": "HASAN ALAKAN", "share_type": "ORDINARY", "shares": 4873, "is_company": false}, {"name": "AMADEUS CAPITAL PARTNERS (NOMINEE) LTD", "share_type": "ORDINARY", "shares": 9746, "is_company": true}, {"name": "IOANNIS ANDREOPOULOS", "share_type": "ORDINARY", "shares": 333716, "is_company": false}, {"name": "ADRIANA BELONIC", "share_type": "ORDINARY", "shares": 2500, "is_company": false}, {"name": "ALEKSANDRA BELONIC", "share_type": "ORDINARY", "shares": 5000, "is_company": false}, {"name": "MEREDITH BENNETT", "share_type": "ORDINARY", "shares": 1949, "is_company": false}, {"name": "ANDREA BUCCA", "share_type": "ORDINARY", "shares": 4873, "is_company": false}, {"name": "CHARLES RONALD CONN III", "share_type": "ORDINARY", "shares": 7797, "is_company": false}, {"name": "DJORDJE DJOKOVIC", "share_type": "ORDINARY", "shares": 0, "is_company": false}, {"name": "VOJISLAV DJURIC", "share_type": "ORDINARY", "shares": 40000, "is_company": false}, {"name": "SRDJAN GRCE", "share_type": "ORDINARY", "shares": 540818, "is_company": false}, {"name": "GYLDENLAKK AS", "share_type": "ORDINARY", "shares": 2923, "is_company": true}, {"name": "ISIZE CORPORATE TRUSTEE LIMITED", "share_type": "ORDINARY", "shares": 158000, "is_company": true}, {"name": "SANIA JEVTIC", "share_type": "ORDINARY", "shares": 5888, "is_company": false}, {"name": "ALEXANDER KOLB", "share_type": "ORDINARY", "shares": 14699, "is_company": false}, {"name": "ANDREW LOFTUS", "share_type": "ORDINARY", "shares": 7837, "is_company": false}, {"name": "STEVEN MARSHALL", "share_type": "ORDINARY", "shares": 17623, "is_company": false}, {"name": "PAUL MASSARA", "share_type": "ORDINARY", "shares": 4873, "is_company": false}, {"name": "DEJAN MILADINOVIC", "share_type": "ORDINARY", "shares": 16500, "is_company": false}, {"name": "MARKO MILOS", "share_type": "ORDINARY", "shares": 38000, "is_company": false}, {"name": "HUGO MARCELO MURIEL ARRIARAN", "share_type": "ORDINARY", "shares": 3858, "is_company": false}, {"name": "OHANA AS", "share_type": "ORDINARY", "shares": 2923, "is_company": true}, {"name": "DHRUV PANDYA", "share_type": "ORDINARY", "shares": 5888, "is_company": false}, {"name": "DUSAN PETROVIC", "share_type": "ORDINARY", "shares": 73000, "is_company": false}, {"name": "PATRICK PICHETTE", "share_type": "ORDINARY", "shares": 4873, "is_company": false}, {"name": "RADKA INVEST AS", "share_type": "ORDINARY", "shares": 1949, "is_company": true}, {"name": "ROBERT SANSOM", "share_type": "ORDINARY", "shares": 7797, "is_company": false}, {"name": "PAOLO SIDERI", "share_type": "ORDINARY", "shares": 974, "is_company": false}, {"name": "MARIA SILVESTRIS", "share_type": "ORDINARY", "shares": 5888, "is_company": false}, {"name": "TAMORER PTY LTD", "share_type": "ORDINARY", "shares": 29238, "is_company": true}, {"name": "TD VEEN AS", "share_type": "ORDINARY", "shares": 38010, "is_company": true}, {"name": "WILLIAM TUNSTALL-PEDOE", "share_type": "ORDINARY", "shares": 3898, "is_company": false}, {"name": "TINA WHITEHOUSE", "share_type": "ORDINARY", "shares": 7837, "is_company": false}, {"name": "COLIN WOODFORD", "share_type": "ORDINARY", "shares": 6862, "is_company": false}, {"name": "A MANAGEMENT AS", "share_type": "SERIES SEED 1 ORDINARY", "shares": 2046, "is_company": true}, {"name": "GYLDENLAKK AS", "share_type": "SERIES SEED 1 ORDINARY", "shares": 1534, "is_company": true}, {"name": "OCTOPUS TITAN VCT PLC", "share_type": "SERIES SEED 1 ORDINARY", "shares": 335249, "is_company": true}, {"name": "OHANA AS", "share_type": "SERIES SEED 1 ORDINARY", "shares": 1534, "is_company": true}, {"name": "RADKA INVEST AS", "share_type": "SERIES SEED 1 ORDINARY", "shares": 1023, "is_company": true}, {"name": "TAMORER PTY LTD", "share_type": "SERIES SEED 1 ORDINARY", "shares": 15346, "is_company": true}, {"name": "TD VEEN AS", "share_type": "SERIES SEED 1 ORDINARY", "shares": 147496, "is_company": true}, {"name": "CHARLES RONALD CONN III", "share_type": "SERIES SEED 2 ORDINARY", "shares": 4092, "is_company": false}, {"name": "CHRISTOPHER JOHN MAIRS", "share_type": "SERIES SEED 2 ORDINARY", "shares": 2554, "is_company": false}, {"name": "OCTOPUS INVESTMENTS NOMINEES LIMITED", "share_type": "SERIES SEED 2 ORDINARY", "shares": 111750, "is_company": true}, {"name": "PATRICK PICHETTE", "share_type": "SERIES SEED 2 ORDINARY", "shares": 2557, "is_company": false}, {"name": "ROBERT SANSOM", "share_type": "SERIES SEED 2 ORDINARY", "shares": 9207, "is_company": false}, {"name": "WILLIAM TUNSTALL-PEDOE", "share_type": "SERIES SEED 2 ORDINARY", "shares": 2046, "is_company": false}]}, "2020-11-26": {"source": "Confirmation Statement", "url": "https://find-and-update.company-information.service.gov.uk/company/10388803/filing-history/MzI4NDQ4MTEyOWFkaXF6a2N4/document?format=pdf&download=0", "received_date": "2020-11-26", "filing_date": "2020-09-21", "update": true, "items": [{"name": "SRDJAN GRCE", "share_type": "ORDINARY", "shares": 540818, "is_company": false}, {"name": "HUGO MARCELO MURIEL ARRIARAN", "share_type": "ORDINARY", "shares": 3858, "is_company": false}, {"name": "DJORDJE DJOKOVIC", "share_type": "ORDINARY", "shares": 94208, "is_company": false}, {"name": "DEJAN MILADINOVIC", "share_type": "ORDINARY", "shares": 16500, "is_company": false}, {"name": "DUSAN PETROVIC", "share_type": "ORDINARY", "shares": 73000, "is_company": false}, {"name": "MARKO MILOS", "share_type": "ORDINARY", "shares": 38000, "is_company": false}, {"name": "VOJISLAV DJURIC", "share_type": "ORDINARY", "shares": 40000, "is_company": false}, {"name": "IOANNIS ANDREOPOULOS", "share_type": "ORDINARY", "shares": 333716, "is_company": false}, {"name": "ALEKSANDRA BELONIC", "share_type": "ORDINARY", "shares": 5000, "is_company": false}, {"name": "ADRIANA BELONIC", "share_type": "ORDINARY", "shares": 2500, "is_company": false}, {"name": "SANIA JEVTIC", "share_type": "ORDINARY", "shares": 5888, "is_company": false}, {"name": "STEVEN MARSHALL", "share_type": "ORDINARY", "shares": 11776, "is_company": false}, {"name": "MARIA SILVESTRIS", "share_type": "ORDINARY", "shares": 5888, "is_company": false}, {"name": "ALEXANDER KOLB", "share_type": "ORDINARY", "shares": 11776, "is_company": false}, {"name": "DHRUV PANDYA", "share_type": "ORDINARY", "shares": 5888, "is_company": false}, {"name": "ANDREW LOFTUS", "share_type": "ORDINARY", "shares": 5888, "is_company": false}, {"name": "COLIN WOODFORD", "share_type": "ORDINARY", "shares": 5888, "is_company": false}, {"name": "TINA WHITEHOUSE", "share_type": "ORDINARY", "shares": 5888, "is_company": false}, {"name": "ISIZE CORPORATE TRUSTEE LIMITED", "share_type": "ORDINARY", "shares": 30000, "is_company": true}]}, "2019-10-07": {"source": "Confirmation Statement", "url": "https://find-and-update.company-information.service.gov.uk/company/10388803/filing-history/MzI0NjE4MjIxN2FkaXF6a2N4/document?format=pdf&download=0", "received_date": "2019-10-07", "filing_date": "2019-09-21", "update": true, "items": [{"name": "SRDJAN GRCE", "share_type": "ORDINARY", "shares": 540818, "is_company": false}, {"name": "HUGO MARCELO MURIEL ARRIARAN", "share_type": "ORDINARY", "shares": 33858, "is_company": false}, {"name": "DJORDJE DJOKOVIC", "share_type": "ORDINARY", "shares": 94208, "is_company": false}, {"name": "DEJAN MILADINOVIC", "share_type": "ORDINARY", "shares": 16500, "is_company": false}, {"name": "DUSAN PETROVIC", "share_type": "ORDINARY", "shares": 73000, "is_company": false}, {"name": "MARKO MILOS", "share_type": "ORDINARY", "shares": 38000, "is_company": false}, {"name": "VOJISLAV DJURIC", "share_type": "ORDINARY", "shares": 40000, "is_company": false}, {"name": "IOANNIS ANDREOPOULOS", "share_type": "ORDINARY", "shares": 333716, "is_company": false}, {"name": "ALEKSANDRA BELONIC", "share_type": "ORDINARY", "shares": 5000, "is_company": false}, {"name": "ADRIANA BELONIC", "share_type": "ORDINARY", "shares": 2500, "is_company": false}, {"name": "SANIA JEVTIC", "share_type": "ORDINARY", "shares": 5888, "is_company": false}, {"name": "STEVEN MARSHALL", "share_type": "ORDINARY", "shares": 11776, "is_company": false}, {"name": "MARIA SILVESTRIS", "share_type": "ORDINARY", "shares": 5888, "is_company": false}, {"name": "ALEXANDER KOLB", "share_type": "ORDINARY", "shares": 11776, "is_company": false}, {"name": "DHRUV PANDYA", "share_type": "ORDINARY", "shares": 5888, "is_company": false}, {"name": "ANDREW LOFTUS", "share_type": "ORDINARY", "shares": 5888, "is_company": false}, {"name": "COLIN WOODFORD", "share_type": "ORDINARY", "shares": 5888, "is_company": false}, {"name": "TINA WHITEHOUSE", "share_type": "ORDINARY", "shares": 5888, "is_company": false}]}, "2018-10-16": {"source": "Confirmation Statement", "url": "https://find-and-update.company-information.service.gov.uk/company/10388803/filing-history/MzIxNzA5NjMxNGFkaXF6a2N4/document?format=pdf&download=0", "received_date": "2018-10-16", "filing_date": "2018-09-21", "update": true, "items": [{"name": "SRDJAN GRCE", "share_type": "ORDINARY", "shares": 436622, "is_company": false}, {"name": "HUGO MARCELO MURIEL ARRIARAN", "share_type": "ORDINARY", "shares": 33858, "is_company": false}, {"name": "DJORDJE DJOKOVIC", "share_type": "ORDINARY", "shares": 302600, "is_company": false}, {"name": "DEJAN MILADINOVIC", "share_type": "ORDINARY", "shares": 15000, "is_company": false}, {"name": "DUSAN PETROVIC", "share_type": "ORDINARY", "shares": 60000, "is_company": false}, {"name": "MARKO MILOS", "share_type": "ORDINARY", "shares": 25000, "is_company": false}, {"name": "VOJISLAV DJURIC", "share_type": "ORDINARY", "shares": 40000, "is_company": false}, {"name": "IOANNIS ANDREOPOULOS", "share_type": "ORDINARY", "shares": 229520, "is_company": false}, {"name": "ALEKSANDRA BELONIC", "share_type": "ORDINARY", "shares": 5000, "is_company": false}]}, "2017-09-29": {"source": "Confirmation Statement", "url": "https://find-and-update.company-information.service.gov.uk/company/10388803/filing-history/MzE4NjYwMjc1NGFkaXF6a2N4/document?format=pdf&download=0", "received_date": "2017-09-29", "filing_date": "2017-09-21", "update": true, "items": [{"name": "SRDJAN GRCE", "share_type": "ORDINARY", "shares": 400000, "is_company": false}, {"name": "HUGO MARCELO MURIEL ARRIARAN", "share_type": "ORDINARY", "shares": 300000, "is_company": false}, {"name": "DJORDJE DJOKOVIC", "share_type": "ORDINARY", "shares": 300000, "is_company": false}, {"name": "DEJAN MILADINOVIC", "share_type": "ORDINARY", "shares": 12500, "is_company": false}, {"name": "DUSAN PETROVIC", "share_type": "ORDINARY", "shares": 50000, "is_company": false}, {"name": "MARKO MILOS", "share_type": "ORDINARY", "shares": 25000, "is_company": false}, {"name": "VOJISLAV DJURIC", "share_type": "ORDINARY", "shares": 40000, "is_company": false}]}, "2016-09-22": {"source": "Incorporation", "url": "https://find-and-update.company-information.service.gov.uk/company/10388803/filing-history/MzE1Nzc4NTQ1N2FkaXF6a2N4/document?format=pdf&download=0", "filing_date": "2016-09-22", "received_date": "2016-09-22", "update": true, "items": [{"name": "SRDJAN GRCE", "address": ["FLAT 3 FALCONET COURT", "flat 3 falconet court", "123 WAPPING HIGH", "STREET", "LONDON", "london share", "UNITED KINGDOM", "E1W 3NX", "arriaran"], "currency": "GBP", "nominal_value": 1.0, "amount_unpaid": 0.0, "amount_paid": 1.0, "share_type": "S  ORDINARY", "shares": 40, "is_company": false}, {"name": "HUGO MARCELO MURIEL", "address": ["11 RUE DES UTTINS", "PESEUX", "SWITZERLAND", "2034", "share"], "currency": "GBP", "nominal_value": 1.0, "amount_unpaid": 0.0, "amount_paid": 1.0, "share_type": "S  ORDINARY", "shares": 30, "is_company": false}, {"name": "DJORDJE DJOKOVIC", "address": ["35 PALACE GARDENS", "terrace", "LONDON", "UNITED KINGDOM", "W8 4SB", "share"], "currency": "GBP", "nominal_value": 1.0, "amount_unpaid": 0.0, "amount_paid": 1.0, "share_type": "S  ORDINARY", "shares": 30, "is_company": false}]}}, "incorporation": {"items": [{"name": "SRDJAN GRCE", "address": ["FLAT 3 FALCONET COURT", "flat 3 falconet court", "123 WAPPING HIGH", "STREET", "LONDON", "london share", "UNITED KINGDOM", "E1W 3NX", "arriaran"], "currency": "GBP", "nominal_value": 1.0, "amount_unpaid": 0.0, "amount_paid": 1.0, "share_type": "S  ORDINARY", "shares": 40, "is_company": false}, {"name": "HUGO MARCELO MURIEL", "address": ["11 RUE DES UTTINS", "PESEUX", "SWITZERLAND", "2034", "share"], "currency": "GBP", "nominal_value": 1.0, "amount_unpaid": 0.0, "amount_paid": 1.0, "share_type": "S  ORDINARY", "shares": 30, "is_company": false}, {"name": "DJORDJE DJOKOVIC", "address": ["35 PALACE GARDENS", "terrace", "LONDON", "UNITED KINGDOM", "W8 4SB", "share"], "currency": "GBP", "nominal_value": 1.0, "amount_unpaid": 0.0, "amount_paid": 1.0, "share_type": "S  ORDINARY", "shares": 30, "is_company": false}], "url": "https://find-and-update.company-information.service.gov.uk/company/10388803/filing-history/MzE1Nzc4NTQ1N2FkaXF6a2N4/document?format=pdf&download=0", "received_date": "2016-09-22", "update": true}, "appointment": {"items": [{"name": "Charles Ronald Conn Iii", "officer_name": "Charles Ronald Conn Iii", "url": "https://find-and-update.company-information.service.gov.uk/officers/_kCUD6uPdwLMYcg1ay2qstFJuFI/appointments", "key": "appointment", "date_of_birth": "1961-08-01", "items": [{"company_name": "Monograph Capital Advisors", "company_id": "UK", "company_status": "Active", "address": "11th Floor, 200 Aldersgate Street, London, United Kingdom, EC1A 4HD", "role": "LLP Designated Member", "status": null, "appointed_on": "2021-03-26", "resigned_on": null, "nationality": null, "residence": "England", "occupation": null}, {"company_name": "Isize Limited", "company_id": "10388803", "company_status": "Active", "address": "9th Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Director", "status": null, "appointed_on": "2020-11-12", "resigned_on": "2022-06-29", "nationality": "American,Canadian", "residence": "England", "occupation": "Company Director"}, {"company_name": "Rhodes House Limited", "company_id": "06539418", "company_status": "Active", "address": "Rhodes House, South Parks Road, Oxford, Oxfordshire, OX1 3RG", "role": "Director", "status": null, "appointed_on": "2013-06-14", "resigned_on": "2019-03-11", "nationality": "American,Canadian", "residence": "England", "occupation": "Company Director"}]}, {"name": "Edward Clifton", "officer_name": "Edward Clifton", "url": "https://find-and-update.company-information.service.gov.uk/officers/xjirzm9boUZhyX_quiHEhWsLrl0/appointments", "key": "appointment", "date_of_birth": "1971-01-01", "items": [{"company_name": "Isize Limited", "company_id": "10388803", "company_status": "Active", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Director", "status": "Active", "appointed_on": "2022-07-20", "resigned_on": null, "nationality": "British", "residence": "United Kingdom", "occupation": "Director"}, {"company_name": "Deontics Limited", "company_id": "08782234", "company_status": "Active", "address": "2/127, High Road, Loughton, Essex, IG10 4LT", "role": "Director", "status": null, "appointed_on": "2015-09-30", "resigned_on": "2019-10-31", "nationality": "British", "residence": "United Kingdom", "occupation": "Company Director"}]}, {"name": "Hugo Marcelo Muriel Arriaran", "officer_name": "Hugo Marcelo Muriel Arriaran", "url": "https://find-and-update.company-information.service.gov.uk/officers/GdP1ZTjt4-Za4T1MsRTb6dYSlFY/appointments", "key": "appointment", "date_of_birth": "1983-10-01", "items": [{"company_name": "Isize Limited", "company_id": "10388803", "company_status": "Active", "address": "3 Falconet Court, 123 Wapping High Street, London, United Kingdom, E1W 3NX", "role": "Director", "status": null, "appointed_on": "2017-02-03", "resigned_on": "2018-11-05", "nationality": "Swiss", "residence": "Switzerland", "occupation": "Computer Scientist"}]}, {"name": "Ioannis Andreopoulos", "officer_name": "Ioannis Andreopoulos", "url": "https://find-and-update.company-information.service.gov.uk/officers/7dK_qGQnZEYGIus3DO_O-QANpho/appointments", "key": "appointment", "date_of_birth": "1977-01-01", "items": [{"company_name": "Isize Corporate Trustee Limited", "company_id": "12617154", "company_status": "Active", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Director", "status": "Active", "appointed_on": "2020-05-21", "resigned_on": null, "nationality": "British", "residence": "England", "occupation": "Director"}, {"company_name": "Isize Limited", "company_id": "10388803", "company_status": "Active", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Director", "status": "Active", "appointed_on": "2018-11-05", "resigned_on": null, "nationality": "British", "residence": "England", "occupation": "Chief Technical Officer"}, {"company_name": "Node-C Limited", "company_id": "10787409", "company_status": "Active", "address": "167a Bathurst Gardens, London, United Kingdom, NW10 5JJ", "role": "Director", "status": "Active", "appointed_on": "2017-05-24", "resigned_on": null, "nationality": "British", "residence": "England", "occupation": "Director"}, {"company_name": "Dithen Limited", "company_id": "09301684", "company_status": "Active", "address": "167a Bathurst Gardens, London, United Kingdom, NW10 5JJ", "role": "Director", "status": "Active", "appointed_on": "2014-11-07", "resigned_on": null, "nationality": "British", "residence": "England", "occupation": "Director"}, {"company_name": "Polytope Technologies Ltd", "company_id": "12075969", "company_status": "Dissolved", "address": "27 Old Gloucester Street, London, United Kingdom, WC1N 3AX", "role": "Director", "status": null, "appointed_on": "2019-06-28", "resigned_on": "2021-05-28", "nationality": "British", "residence": "England", "occupation": "Professor"}]}, {"name": "Marko Milos", "officer_name": "Marko Milos", "url": "https://find-and-update.company-information.service.gov.uk/officers/3oUbsQ1L6sfBFt4cR00eiL38o8M/appointments", "key": "appointment", "date_of_birth": "1978-04-01", "items": [{"company_name": "Isize Limited", "company_id": "10388803", "company_status": "Active", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Director", "status": "Active", "appointed_on": "2017-03-27", "resigned_on": null, "nationality": "English", "residence": "England", "occupation": "Banker"}]}, {"name": "Oakwood Corporate Secretary Limited", "officer_name": "Oakwood Corporate Secretary Limited", "url": "https://find-and-update.company-information.service.gov.uk/officers/qkCSxkXy04LQ3YY6ZfkveKpkaCM/appointments", "key": "appointment", "date_of_birth": null, "items": [{"company_name": "Gordon Midco Limited", "company_id": "12297257", "company_status": "Dissolved", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2019-11-04", "resigned_on": "2019-12-19", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Sherborne Sensors Limited", "company_id": "04477365", "company_status": "Active", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": "Active", "appointed_on": "2015-06-08", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "John D Wood & Co.", "company_id": "RESIDENTIAL & AGRICULTURAL", "company_status": "Active", "address": "3rd, Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2013-01-29", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Integenx Limited", "company_id": "07414702", "company_status": "Active", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": "Active", "appointed_on": "2018-07-18", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Allsports.Co.Uk Limited", "company_id": "01000415", "company_status": "Active", "address": "3rd, Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": "Active", "appointed_on": "2010-07-01", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Ge Capital Solutions Europe Limited", "company_id": "01596085", "company_status": "Dissolved", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2019-05-16", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Countrywide Estate Agents Nominees Limited", "company_id": "01875079", "company_status": "Dissolved", "address": "3rd, Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2012-12-12", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Ge Energy Power Conversion Uk Holdings Limited", "company_id": "06702141", "company_status": "Active", "address": "3rd, Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": "Active", "appointed_on": "2012-09-14", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Oxoid Senior Holdings Limited", "company_id": "05281667", "company_status": "Active", "address": "3rd, Floor 1 Ashley Road, Altrincham, Cheshire, WA14 2DT", "role": "Secretary", "status": "Active", "appointed_on": "2009-11-01", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Ge", "company_id": "SIGMA", "company_status": "Liquidation", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2019-07-23", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Mintun 2 Limited", "company_id": "04632319", "company_status": "Active", "address": "3rd, Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": "Active", "appointed_on": "2011-04-01", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Debt Settlement Solutions Limited", "company_id": "07996790", "company_status": "Dissolved", "address": "3rd, Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2013-08-08", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Spacex-Uk Ltd", "company_id": "10723064", "company_status": "Dissolved", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2017-04-12", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Rossendales High Court Enforcement Limited", "company_id": "07180434", "company_status": "Dissolved", "address": "3rd, Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2013-03-21", "resigned_on": "2013-11-08", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Big Dog Marketing Limited", "company_id": "07538163", "company_status": "Dissolved", "address": "3rd, Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2013-07-29", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Mmw Europe Ltd", "company_id": "09081977", "company_status": "Active", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2014-10-01", "resigned_on": "2015-06-10", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Thermo Measurement Ltd", "company_id": "03284221", "company_status": "Liquidation", "address": "3rd, Floor 1 Ashley Road, Altrincham, Cheshire, WA14 2DT", "role": "Secretary", "status": "Active", "appointed_on": "2009-11-01", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Richard Cochrane Iip Settlement 2003 Limited", "company_id": "04938349", "company_status": "Active", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2014-07-29", "resigned_on": "2019-04-30", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Guy Barnett Iip Settlement 2003 Limited", "company_id": "04941868", "company_status": "Dissolved", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2014-07-29", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Merchant Maintenance Limited", "company_id": "SC378306", "company_status": "Active", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": "Active", "appointed_on": "2014-07-08", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Ashton Burkinshaw", "company_id": "FRANCHISING", "company_status": "Active", "address": "3rd, Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2013-03-20", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Industrium Group Limited", "company_id": "12433678", "company_status": "Active", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": "Active", "appointed_on": "2020-01-30", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Gerard Whitehouse Iip Settlement 2003 Limited", "company_id": "04941837", "company_status": "Dissolved", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2014-07-29", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "G.E. S.P.S. Pension Trustees Ltd.", "company_id": "02669309", "company_status": "Dissolved", "address": "3rd, Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2012-11-01", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Entwistle Green Limited", "company_id": "05255190", "company_status": "Active", "address": "3rd, Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": "Active", "appointed_on": "2013-02-11", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Peggy Midco Limited", "company_id": "11441437", "company_status": "Active", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2018-06-29", "resigned_on": "2018-07-09", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Wikipad International Limited", "company_id": "05277603", "company_status": "Dissolved", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2018-10-16", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Cadihayes Services", "company_id": "NO 3", "company_status": "Active", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2016-07-01", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Portalplanquest Limited", "company_id": "09400439", "company_status": "Active", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": "Active", "appointed_on": "2020-12-21", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "The Quick Cash Club Limited", "company_id": "08078991", "company_status": "Dissolved", "address": "3rd, Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2013-08-28", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Glyde Limited", "company_id": "06931699", "company_status": "Active", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2019-01-24", "resigned_on": "2021-06-02", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Slatershelfco 261 Limited", "company_id": "02815332", "company_status": "Active", "address": "3rd, Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": "Active", "appointed_on": "2011-01-24", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Ge Power Resources Management Ltd", "company_id": "00540682", "company_status": "Dissolved", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": null, "appointed_on": "2020-05-15", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Seavacations Uk Limited", "company_id": "03633566", "company_status": "Active", "address": "3rd, Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": "Active", "appointed_on": "2009-11-01", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Retail Finance Intelligence Limited", "company_id": "06780168", "company_status": "Active", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Secretary", "status": "Active", "appointed_on": "2020-11-12", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}]}, {"name": "Ohs Secretaries Limited", "officer_name": "Ohs Secretaries Limited", "url": "https://find-and-update.company-information.service.gov.uk/officers/8DTKf650Reo5ubQ17kwO3hBSxt4/appointments", "key": "appointment", "date_of_birth": null, "items": [{"company_name": "Johnson Controls Air Conditioning Europe Limited", "company_id": "09711144", "company_status": "Active", "address": "9th Floor, 107 Cheapside, London, England, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2016-05-01", "resigned_on": "2017-03-07", "nationality": null, "residence": null, "occupation": null}, {"company_name": "New Relic Uk Limited", "company_id": "09068859", "company_status": "Active", "address": "9th Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": "Active", "appointed_on": "2017-06-06", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Vchain Tech Limited", "company_id": "09921460", "company_status": "Dissolved", "address": "9th Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2015-12-17", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Mopapp Limited", "company_id": "07649622", "company_status": "Dissolved", "address": "9th, Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2011-05-26", "resigned_on": "2012-09-10", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Zatili Limited", "company_id": "11193215", "company_status": "Active", "address": "9th Floor, 107 Cheapside, London, England, England, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2018-02-07", "resigned_on": "2018-03-26", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Transfergo Ltd", "company_id": "07914165", "company_status": "Active", "address": "9th, Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": "Active", "appointed_on": "2013-02-18", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Reyn Limited", "company_id": "11136060", "company_status": "Dissolved", "address": "9th Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2018-01-05", "resigned_on": "2021-08-10", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Paynama Ltd", "company_id": "08445036", "company_status": "Dissolved", "address": "9th, Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2013-03-14", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Levi Strauss", "company_id": "U.K.", "company_status": "Active", "address": "107 Cheapside, London, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2009-11-03", "resigned_on": "2015-06-30", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Voly Limited", "company_id": "10412388", "company_status": "Active", "address": "9th Floor, 107 Cheapside, London, England, England, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2019-09-17", "resigned_on": "2021-04-13", "nationality": null, "residence": null, "occupation": null}, {"company_name": "The Currency Cloud Limited", "company_id": "06323311", "company_status": "Active", "address": "9th Floor, 107 Cheapside, London, England, England, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2020-12-03", "resigned_on": "2021-12-20", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Parthian Partners", "company_id": "UK", "company_status": "Active", "address": "9th Floor, 107 Cheapside, London, England, England, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2019-06-18", "resigned_on": "2020-08-07", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Tossed Limited", "company_id": "07186955", "company_status": "Liquidation", "address": "9th, Floor, 107 Cheapside, London, Gb, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2010-03-11", "resigned_on": "2010-04-06", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Tesorx Uk, Ltd", "company_id": "09349303", "company_status": "Dissolved", "address": "9th Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2014-12-10", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Legal Nemesis Limited", "company_id": "10288888", "company_status": "Dissolved", "address": "9th Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2016-07-20", "resigned_on": "2018-08-10", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Ebay Kta", "company_id": "UK", "company_status": "Active", "address": "9th Floor, 107 Cheapside, London, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2011-02-04", "resigned_on": "2014-05-12", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Speakeasy Tech Uk Limited", "company_id": "08879564", "company_status": "Dissolved", "address": "9th Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2015-05-28", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Sumerian Partners Limited", "company_id": "08363117", "company_status": "Active", "address": "9th Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2015-09-29", "resigned_on": "2017-01-09", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Uk Investments Management Limited", "company_id": "05476867", "company_status": "Dissolved", "address": "9th, Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2010-11-15", "resigned_on": "2017-05-30", "nationality": null, "residence": null, "occupation": null}, {"company_name": "D.Labs", "company_id": "UK", "company_status": "Dissolved", "address": "9th, Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2011-10-03", "resigned_on": "2014-10-01", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Peer2Peer Systems Ltd", "company_id": "06053100", "company_status": "Dissolved", "address": "9th Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2015-04-22", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Remote-Learner Uk Limited", "company_id": "07519978", "company_status": "Dissolved", "address": "9th Floor, 107 Cheapside, London, England, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2016-07-27", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Wirex Limited", "company_id": "09334596", "company_status": "Active", "address": "9th Floor, 107 Cheapside, London, England, England, EC2V 6DN", "role": "Secretary", "status": "Active", "appointed_on": "2020-10-14", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Extrinsica Global Holdings Limited", "company_id": "10748417", "company_status": "Active", "address": "9th Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2020-02-03", "resigned_on": "2023-06-02", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Health Innovators Ltd", "company_id": "12317957", "company_status": "Active", "address": "9th Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2019-11-15", "resigned_on": "2021-11-16", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Respoke Limited", "company_id": "08265501", "company_status": "Active", "address": "9th Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": "Active", "appointed_on": "2020-01-10", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Q Doctor International Limited", "company_id": "12545831", "company_status": "Dissolved", "address": "9th Floor, 107 Cheapside, London, England, England, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2020-04-03", "resigned_on": "2021-07-27", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Infinitemonkeys.Ai Limited", "company_id": "10857549", "company_status": "Dissolved", "address": "9th Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2017-07-10", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "The Copyrights Group Limited", "company_id": "01819018", "company_status": "Active", "address": "9th Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2017-04-05", "resigned_on": "2019-12-05", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Stockflare Analytics Ltd", "company_id": "08522490", "company_status": "Active", "address": "9th, Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2013-05-09", "resigned_on": "2014-12-30", "nationality": null, "residence": null, "occupation": null}, {"company_name": "S94 Ltd.", "company_id": "11935713", "company_status": "Liquidation", "address": "9th Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": "Active", "appointed_on": "2019-04-09", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Fat Llama Ltd", "company_id": "09927338", "company_status": "Active", "address": "9th Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2017-05-15", "resigned_on": "2020-07-09", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Corventis Limited", "company_id": "06673415", "company_status": "Dissolved", "address": "Tower 42  Level 35, 25 Old Broad Street, London, EC2N 1HQ", "role": "Secretary", "status": null, "appointed_on": "2009-04-15", "resigned_on": "2010-06-01", "nationality": null, "residence": null, "occupation": null}, {"company_name": "Oorja Development Solutions Limited", "company_id": "10352021", "company_status": "Active", "address": "9th Floor, 107 Cheapside, London, United Kingdom, EC2V 6DN", "role": "Secretary", "status": "Active", "appointed_on": "2016-11-07", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}, {"company_name": "Txttools Limited", "company_id": "07065732", "company_status": "Dissolved", "address": "9th Floor, 107 Cheapside, London, England, EC2V 6DN", "role": "Secretary", "status": null, "appointed_on": "2016-07-27", "resigned_on": null, "nationality": null, "residence": null, "occupation": null}]}, {"name": "Simon Paul", "officer_name": "Simon Paul King", "url": "https://find-and-update.company-information.service.gov.uk/officers/K1hjAdUc1lmw5-PCEBzC78AdUeQ/appointments", "key": "appointment", "date_of_birth": "1984-05-01", "items": [{"company_name": "Surrey Nanosystems Limited", "company_id": "03781545", "company_status": "Active", "address": "East Side Business Park, Beach Road, Newhaven, England, BN9 0FB", "role": "Director", "status": "Active", "appointed_on": "2022-11-01", "resigned_on": null, "nationality": "British", "residence": "United Kingdom", "occupation": "Financial Services"}, {"company_name": "Comind Technologies Limited", "company_id": "11729526", "company_status": "Active", "address": "Octopus Investmnets, 33 Holborn, London, England, EC1N 2HT", "role": "Director", "status": "Active", "appointed_on": "2021-04-01", "resigned_on": null, "nationality": "British", "residence": "United Kingdom", "occupation": "Financial Services"}, {"company_name": "Isize Limited", "company_id": "10388803", "company_status": "Active", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Director", "status": "Active", "appointed_on": "2021-04-01", "resigned_on": null, "nationality": "British", "residence": "United Kingdom", "occupation": "Director"}, {"company_name": "Orbital Express Launch Limited", "company_id": "09580714", "company_status": "Active", "address": "1st Floor, One Suffolk Way, Sevenoaks, Kent, England, TN13 1YL", "role": "Director", "status": "Active", "appointed_on": "2020-12-04", "resigned_on": null, "nationality": "British", "residence": "United Kingdom", "occupation": "Investment Manager"}, {"company_name": "Smartkem, Inc", "company_id": "FC038015", "company_status": "Active", "address": "Manchester Technology Center, Hexagon Tower, Delaunays Road, Blackley, England, M9 8GQ", "role": "Director", "status": null, "appointed_on": "2021-02-23", "resigned_on": "2022-06-30", "nationality": "British", "residence": "United Kingdom", "occupation": "Director"}]}, {"name": "Srdjan Grce", "officer_name": "Srdjan Grce", "url": "https://find-and-update.company-information.service.gov.uk/officers/5jfbbnRTFU8OF_-nOcvN2wcMReo/appointments", "key": "appointment", "date_of_birth": "1977-10-01", "items": [{"company_name": "Isize Corporate Trustee Limited", "company_id": "12617154", "company_status": "Active", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Director", "status": "Active", "appointed_on": "2020-05-21", "resigned_on": null, "nationality": "British", "residence": "United Kingdom", "occupation": "Director"}, {"company_name": "Miledean Ltd", "company_id": "11388146", "company_status": "Dissolved", "address": "Flat 3 Falconet Court, 123 Wapping High Street, London, United Kingdom, E1W 3NX", "role": "Director", "status": null, "appointed_on": "2018-05-30", "resigned_on": null, "nationality": "British", "residence": "United Kingdom", "occupation": "Director"}, {"company_name": "Isize Limited", "company_id": "10388803", "company_status": "Active", "address": "3rd Floor, 1 Ashley Road, Altrincham, Cheshire, United Kingdom, WA14 2DT", "role": "Director", "status": "Active", "appointed_on": "2016-09-22", "resigned_on": null, "nationality": "British", "residence": "United Kingdom", "occupation": "Founder & Ceo"}, {"company_name": "Fusion Artworks Ltd", "company_id": "09885320", "company_status": "Dissolved", "address": "Falconet Court, Flat 3 Falconet Court, 123 Wapping High Street, London, London, United Kingdom, E1W 3NX", "role": "Director", "status": null, "appointed_on": "2015-11-24", "resigned_on": null, "nationality": "British", "residence": "United Kingdom", "occupation": "Art"}, {"company_name": "Falconet Court Freehold Limited", "company_id": "03901386", "company_status": "Active", "address": "Flat 3, Falconet Court, 123 Wapping High Street, London, E1W 3NX", "role": "Director", "status": null, "appointed_on": "2007-09-27", "resigned_on": "2012-06-07", "nationality": "British", "residence": "United Kingdom", "occupation": "None"}]}]}}, "source": "companieshouse"}"""
companieshouse_sample = ujson.loads(companieshouse_sample)
crunchbase_sample = """{"properties": {"company_type": "for_profit", "aliases": ["iSIZE"], "rank_delta_d30": -0.5, "image_url": "https://res.cloudinary.com/crunchbase-production/image/upload/xmuwkvi2citoghycs1tu", "founded_on": {"value": "2016-09-22", "precision": "day"}, "website": {"value": "https://www.isize.co"}, "name": "iSIZE Technologies", "equity_funding_total": {"value": 5815000, "currency": "GBP", "value_usd": 7942415}, "rank_principal": 37268, "permalink": "isize-technologies", "facet_ids": ["company", "rank"], "identifier": {"uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "image_id": "xmuwkvi2citoghycs1tu", "ipo_status": "private", "founder_identifiers": [{"uuid": "3730d28c-bbad-4f0a-9144-807a73ccc753", "value": "Sergio Grce", "image_id": "tbybdokidnthuq2e9u5l", "permalink": "sergio-grce", "entity_def_id": "person"}, {"uuid": "c153e92d-1e37-4bb4-aabd-7ff10862f97e", "value": "Yiannis Andreopoulos", "image_id": "kfqafjauihvf5qpwnim3", "permalink": "yiannis-andreopoulos", "entity_def_id": "person"}], "description": "iSIZE Technologies is a company to delivers deep neural networks for data-driven video preprocessing in order to provide significant bitrate savings over any standard encoder, or significant quality improvement at the same bitrate. This will also allow for other advanced ai services like automated tagging and recognition to run seamlessly in conjunction with live content delivery.", "category_groups": [{"uuid": "e5514a50-8200-7f6b-de87-b07990670800", "value": "Artificial Intelligence", "permalink": "artificial-intelligence-e551", "entity_def_id": "category_group"}, {"uuid": "d5f76b18-83d5-c86d-25cb-facca572d71a", "value": "Content and Publishing", "permalink": "content-and-publishing", "entity_def_id": "category_group"}, {"uuid": "701eef4f-18c1-4aff-b550-caf732cd575f", "value": "Data and Analytics", "permalink": "data-and-analytics", "entity_def_id": "category_group"}, {"uuid": "4fe3f3ac-e522-5889-7477-c1b6d6663710", "value": "Information Technology", "permalink": "information-technology-4fe3", "entity_def_id": "category_group"}, {"uuid": "3bd75a3d-986a-9911-b145-f944d7d6d4c6", "value": "Media and Entertainment", "permalink": "media-and-entertainment-3bd7", "entity_def_id": "category_group"}, {"uuid": "285e29fc-8f70-bf00-1749-9e94158f64f4", "value": "Science and Engineering", "permalink": "science-and-engineering", "entity_def_id": "category_group"}, {"uuid": "85b6bca9-930a-11bc-a608-a513b76fb637", "value": "Software", "permalink": "software-85b6", "entity_def_id": "category_group"}, {"uuid": "d4574740-e74d-5087-fc0f-8244b32d6b99", "value": "Video", "permalink": "video-d457", "entity_def_id": "category_group"}], "linkedin": {"value": "https://www.linkedin.com/company/isize.co/"}, "uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "short_description": "iSIZE Technologies is a company to delivers deep neural networks for data-driven video preprocessing.", "num_current_positions": 3, "num_investors": 9, "operating_status": "active", "num_lead_investors": 2, "rank_org": 37267, "facebook": {"value": "https://en-gb.facebook.com/isize.co/"}, "num_employees_enum": "c_00011_00050", "status": "operating", "funding_total": {"value": 5815000, "currency": "GBP", "value_usd": 7942415}, "num_funding_rounds": 4, "last_equity_funding_type": "series_a", "investor_identifiers": [{"uuid": "248e28ee-db49-a613-6a5e-6120f68456ad", "value": "Creative Destruction Lab (CDL)", "image_id": "v1427491020/lkxian3nnkephyzeagws.jpg", "permalink": "creative-destruction-lab", "entity_def_id": "organization"}, {"uuid": "c901d65e-c346-e1a8-7a87-58dd6ca991f9", "value": "Octopus Ventures", "image_id": "kl5vvz6vbs6qzn32fkyz", "permalink": "octopus-ventures", "entity_def_id": "organization"}, {"uuid": "2d90af63-9abb-fa96-03cd-a5c7e897579c", "value": "William Tunstall-Pedoe", "image_id": "kuhdy2jn93u8i5zmngd5", "permalink": "william-tunstall-pedoe", "entity_def_id": "person"}, {"uuid": "8d851678-9164-e490-a3ae-7abcdb3f2287", "value": "Patrick Pichette", "image_id": "edngsa68mbsaf5or6j9b", "permalink": "patrick-pichette", "entity_def_id": "person"}, {"uuid": "0cf953b7-00bb-e607-e775-f781b1b7cf51", "value": "TD Veen", "image_id": "v1431928192/dn2s3anncilsctiyqsxk.png", "permalink": "td-veen", "entity_def_id": "organization"}], "last_funding_type": "series_a", "categories": [{"uuid": "c4d8caf3-5fe7-359b-f9f2-2d708378e4ee", "value": "Artificial Intelligence", "permalink": "artificial-intelligence", "entity_def_id": "category"}, {"uuid": "dbca89fa-f083-5438-b4ad-d3fdeceb78e7", "value": "Information Technology", "permalink": "information-technology-dbca", "entity_def_id": "category"}, {"uuid": "5ea0cdb7-c9a6-47fc-50f8-c9b0fac04863", "value": "Machine Learning", "permalink": "machine-learning", "entity_def_id": "category"}, {"uuid": "c08b5441-a05b-9777-b7a6-012728caddd9", "value": "Software", "permalink": "software", "entity_def_id": "category"}, {"uuid": "c6adf822-7fa3-c0b3-5188-556e08d72128", "value": "Video", "permalink": "video", "entity_def_id": "category"}, {"uuid": "e5445f23-204d-6d79-5309-2c17f1e66d78", "value": "Video on Demand", "permalink": "video-on-demand-e544", "entity_def_id": "category"}, {"uuid": "b1b3b2d7-85ed-2cb1-fc60-3e2b6a3b5ddd", "value": "Video Streaming", "permalink": "video-streaming", "entity_def_id": "category"}], "legal_name": "iSIZE Limited.", "created_at": "2018-08-28T10:26:30Z", "location_identifiers": [{"uuid": "aad17950-576b-8c44-8fd4-f44dbeb59220", "value": "London", "permalink": "london-england", "entity_def_id": "location", "location_type": "city"}, {"uuid": "79eb923b-9e93-e0db-2fe0-75f0c430c2cb", "value": "England", "permalink": "england-united-kingdom", "entity_def_id": "location", "location_type": "region"}, {"uuid": "a30e342c-1742-6b1c-66e9-461de680e54b", "value": "United Kingdom", "permalink": "united-kingdom", "entity_def_id": "location", "location_type": "country"}, {"uuid": "6106f5dc-823e-5da8-40d7-51612c0b2c4e", "value": "Europe", "permalink": "europe", "entity_def_id": "location", "location_type": "continent"}], "num_articles": 119, "last_funding_at": "2021-05-06", "num_event_appearances": 5, "twitter": {"value": "https://twitter.com/isize_co"}, "rank_delta_d90": -3.8, "website_url": "https://www.isize.co", "updated_at": "2022-07-19T01:21:50Z", "last_equity_funding_total": {"value": 4500000.0, "currency": "GBP", "value_usd": 6252752.948173015}, "entity_def_id": "organization", "contact_email": "info@isize.co", "funding_stage": "early_stage_venture", "rank_delta_d7": -0.1, "last_funding_total": {"value": 4500000.0, "currency": "GBP", "value_usd": 6252752.948173015}, "num_founders": 2}, "cards": {"participated_investments": [], "headquarters_address": [{"headquartered_organization_identifier": {"uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "identifier": {"uuid": "218e7f36-6081-48cf-8621-4090a892dd9d", "value": "Headquarters", "entity_def_id": "address"}, "street_1": "3 Falconet Court", "country_code": "GB", "postal_code": "E1W 3NX", "street_2": "123 Wapping High Street", "region_code": "H9", "created_at": "2018-08-28T10:26:30Z", "location_identifiers": [{"uuid": "aad17950-576b-8c44-8fd4-f44dbeb59220", "value": "London", "permalink": "london-england", "entity_def_id": "location", "location_type": "city"}, {"uuid": "79eb923b-9e93-e0db-2fe0-75f0c430c2cb", "value": "England", "permalink": "england-united-kingdom", "entity_def_id": "location", "location_type": "region"}, {"uuid": "a30e342c-1742-6b1c-66e9-461de680e54b", "value": "United Kingdom", "permalink": "united-kingdom", "entity_def_id": "location", "location_type": "country"}, {"uuid": "6106f5dc-823e-5da8-40d7-51612c0b2c4e", "value": "Europe", "permalink": "europe", "entity_def_id": "location", "location_type": "continent"}], "updated_at": "2018-08-28T10:26:30Z"}], "child_organizations": [], "raised_investments": [{"is_lead_investor": true, "identifier": {"uuid": "08d98362-458a-4340-813e-edc8ba09c212", "value": "Octopus Ventures investment in Series A - iSIZE Technologies", "permalink": "octopus-ventures-invested-in-isize-technologies-series-a--b5bba780--08d98362", "entity_def_id": "investment"}, "partner_identifiers": [{"uuid": "a982c120-c0d8-6745-30ff-8e42f3bace05", "value": "Simon King", "image_id": "v1498115333/piewmmrycy625vn57x4y.png", "permalink": "simon-king-2", "entity_def_id": "person"}], "investor_stage": ["early_stage_venture", "late_stage_venture", "seed"], "announced_on": "2021-05-06", "organization_identifier": {"uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "funding_round_identifier": {"uuid": "b5bba780-737a-492a-98e3-9298e2a4266e", "value": "Series A - iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies-series-a--b5bba780", "entity_def_id": "funding_round"}, "funding_round_money_raised": {"value": 4500000, "currency": "GBP", "value_usd": 6252753}, "funding_round_investment_type": "series_a", "investor_type": ["venture_capital"], "created_at": "2021-05-06T19:44:15Z", "updated_at": "2021-05-06T19:44:15Z", "investor_identifier": {"uuid": "c901d65e-c346-e1a8-7a87-58dd6ca991f9", "value": "Octopus Ventures", "image_id": "kl5vvz6vbs6qzn32fkyz", "permalink": "octopus-ventures", "entity_def_id": "organization"}}, {"is_lead_investor": false, "identifier": {"uuid": "2f4e2458-2549-419a-b67a-8d3cfa98d1ce", "value": "TD Veen investment in Series A - iSIZE Technologies", "permalink": "td-veen-invested-in-isize-technologies-series-a--b5bba780--2f4e2458", "entity_def_id": "investment"}, "announced_on": "2021-05-06", "organization_identifier": {"uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "funding_round_identifier": {"uuid": "b5bba780-737a-492a-98e3-9298e2a4266e", "value": "Series A - iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies-series-a--b5bba780", "entity_def_id": "funding_round"}, "funding_round_money_raised": {"value": 4500000, "currency": "GBP", "value_usd": 6252753}, "funding_round_investment_type": "series_a", "investor_type": ["venture_capital"], "created_at": "2021-05-06T19:44:15Z", "updated_at": "2021-05-06T19:44:15Z", "investor_identifier": {"uuid": "0cf953b7-00bb-e607-e775-f781b1b7cf51", "value": "TD Veen", "image_id": "v1431928192/dn2s3anncilsctiyqsxk.png", "permalink": "td-veen", "entity_def_id": "organization"}}, {"identifier": {"uuid": "12e6a89e-c8eb-4876-919d-824e922f9a09", "value": "Patrick Pichette investment in Seed Round - iSIZE Technologies", "permalink": "patrick-pichette-invested-in-isize-technologies-seed--423015a6--12e6a89e", "entity_def_id": "investment"}, "investor_stage": ["seed"], "announced_on": "2020-05-11", "organization_identifier": {"uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "funding_round_identifier": {"uuid": "423015a6-011f-4e54-8cbc-c9fe0c2e2298", "value": "Seed Round - iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies-seed--423015a6", "entity_def_id": "funding_round"}, "funding_round_money_raised": {"value": 740000, "currency": "GBP", "value_usd": 912549}, "funding_round_investment_type": "seed", "investor_type": ["investment_partner", "angel"], "created_at": "2020-05-13T05:42:06Z", "updated_at": "2020-05-13T05:42:06Z", "investor_identifier": {"uuid": "8d851678-9164-e490-a3ae-7abcdb3f2287", "value": "Patrick Pichette", "image_id": "edngsa68mbsaf5or6j9b", "permalink": "patrick-pichette", "entity_def_id": "person"}}, {"identifier": {"uuid": "26bf92f8-9744-4337-b61f-fb7edc2b47bd", "value": "Charles Conn investment in Seed Round - iSIZE Technologies", "permalink": "charles-conn-invested-in-isize-technologies-seed--423015a6--26bf92f8", "entity_def_id": "investment"}, "investor_stage": ["seed"], "announced_on": "2020-05-11", "organization_identifier": {"uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "funding_round_identifier": {"uuid": "423015a6-011f-4e54-8cbc-c9fe0c2e2298", "value": "Seed Round - iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies-seed--423015a6", "entity_def_id": "funding_round"}, "funding_round_money_raised": {"value": 740000, "currency": "GBP", "value_usd": 912549}, "funding_round_investment_type": "seed", "investor_type": ["investment_partner", "angel"], "created_at": "2020-05-13T20:34:12Z", "updated_at": "2020-05-13T20:34:12Z", "investor_identifier": {"uuid": "ba2ab90f-1b3a-d5e4-8fc4-8953c9f6313a", "value": "Charles Conn", "image_id": "v1456479977/i0ik5tsrepm6jk9k8aic.jpg", "permalink": "charles-conn", "entity_def_id": "person"}}, {"identifier": {"uuid": "5b70b410-2380-4d87-8c37-96674fab021c", "value": "Paul Massara investment in Seed Round - iSIZE Technologies", "permalink": "paul-massara-aaab-invested-in-isize-technologies-seed--423015a6--5b70b410", "entity_def_id": "investment"}, "investor_stage": ["early_stage_venture", "seed"], "announced_on": "2020-05-11", "organization_identifier": {"uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "funding_round_identifier": {"uuid": "423015a6-011f-4e54-8cbc-c9fe0c2e2298", "value": "Seed Round - iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies-seed--423015a6", "entity_def_id": "funding_round"}, "funding_round_money_raised": {"value": 740000, "currency": "GBP", "value_usd": 912549}, "funding_round_investment_type": "seed", "investor_type": ["angel"], "created_at": "2020-05-13T20:49:37Z", "updated_at": "2020-05-13T20:49:37Z", "investor_identifier": {"uuid": "95a1f09e-6b4f-4aa9-8066-2215c949aaab", "value": "Paul Massara", "image_id": "zefgqo0adxsaeq044bkw", "permalink": "paul-massara-aaab", "entity_def_id": "person"}}, {"is_lead_investor": true, "identifier": {"uuid": "9a24b78a-822c-4669-9e94-1ed4a2d707a5", "value": "TD Veen investment in Seed Round - iSIZE Technologies", "permalink": "td-veen-invested-in-isize-technologies-seed--423015a6--9a24b78a", "entity_def_id": "investment"}, "announced_on": "2020-05-11", "organization_identifier": {"uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "funding_round_identifier": {"uuid": "423015a6-011f-4e54-8cbc-c9fe0c2e2298", "value": "Seed Round - iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies-seed--423015a6", "entity_def_id": "funding_round"}, "funding_round_money_raised": {"value": 740000, "currency": "GBP", "value_usd": 912549}, "funding_round_investment_type": "seed", "investor_type": ["venture_capital"], "created_at": "2020-05-13T05:32:29Z", "updated_at": "2020-05-13T05:32:29Z", "investor_identifier": {"uuid": "0cf953b7-00bb-e607-e775-f781b1b7cf51", "value": "TD Veen", "image_id": "v1431928192/dn2s3anncilsctiyqsxk.png", "permalink": "td-veen", "entity_def_id": "organization"}}, {"identifier": {"uuid": "b3ae9107-a0d5-495c-9113-359b6b37a168", "value": "Robert Sansom investment in Seed Round - iSIZE Technologies", "permalink": "robert-sansom-invested-in-isize-technologies-seed--423015a6--b3ae9107", "entity_def_id": "investment"}, "investor_stage": ["seed"], "announced_on": "2020-05-11", "organization_identifier": {"uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "funding_round_identifier": {"uuid": "423015a6-011f-4e54-8cbc-c9fe0c2e2298", "value": "Seed Round - iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies-seed--423015a6", "entity_def_id": "funding_round"}, "funding_round_money_raised": {"value": 740000, "currency": "GBP", "value_usd": 912549}, "funding_round_investment_type": "seed", "investor_type": ["investment_partner", "angel"], "created_at": "2020-05-13T05:45:25Z", "updated_at": "2020-05-13T05:45:25Z", "investor_identifier": {"uuid": "69294b81-b677-7b37-975a-9640bc1d938a", "value": "Robert Sansom", "image_id": "v1469005050/a7xv7rcazqrtpnirtnlh.png", "permalink": "robert-sansom", "entity_def_id": "person"}}, {"identifier": {"uuid": "f192c55e-78e9-47ac-9b0d-fa0a98a6bf72", "value": "William Tunstall-Pedoe investment in Seed Round - iSIZE Technologies", "permalink": "william-tunstall-pedoe-invested-in-isize-technologies-seed--423015a6--f192c55e", "entity_def_id": "investment"}, "investor_stage": ["seed"], "announced_on": "2020-05-11", "organization_identifier": {"uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "funding_round_identifier": {"uuid": "423015a6-011f-4e54-8cbc-c9fe0c2e2298", "value": "Seed Round - iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies-seed--423015a6", "entity_def_id": "funding_round"}, "funding_round_money_raised": {"value": 740000, "currency": "GBP", "value_usd": 912549}, "funding_round_investment_type": "seed", "investor_type": ["angel"], "created_at": "2020-05-13T05:44:10Z", "updated_at": "2020-05-13T05:44:10Z", "investor_identifier": {"uuid": "2d90af63-9abb-fa96-03cd-a5c7e897579c", "value": "William Tunstall-Pedoe", "image_id": "kuhdy2jn93u8i5zmngd5", "permalink": "william-tunstall-pedoe", "entity_def_id": "person"}}, {"identifier": {"uuid": "9297888e-54c6-4954-a71d-7a2e3c8e8609", "value": "Creative Destruction Lab (CDL) investment in Pre Seed Round - iSIZE Technologies", "permalink": "creative-destruction-lab-invested-in-isize-technologies-pre-seed--9b03576f--9297888e", "entity_def_id": "investment"}, "investor_stage": ["early_stage_venture", "seed"], "announced_on": "2019-02-04", "organization_identifier": {"uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "funding_round_identifier": {"uuid": "9b03576f-698a-4e94-a4ee-7f29ddb63456", "value": "Pre Seed Round - iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies-pre-seed--9b03576f", "entity_def_id": "funding_round"}, "funding_round_money_raised": {"value": 200000, "currency": "GBP", "value_usd": 260543}, "funding_round_investment_type": "pre_seed", "investor_type": ["entrepreneurship_program", "university_program"], "created_at": "2020-07-27T13:28:43Z", "updated_at": "2020-07-27T13:28:43Z", "investor_identifier": {"uuid": "248e28ee-db49-a613-6a5e-6120f68456ad", "value": "Creative Destruction Lab (CDL)", "image_id": "v1427491020/lkxian3nnkephyzeagws.jpg", "permalink": "creative-destruction-lab", "entity_def_id": "organization"}}, {"identifier": {"uuid": "c8bb32d4-75d3-474b-8285-a0da0ca6b479", "value": "Multiple Angels investment in Pre Seed Round - iSIZE Technologies", "permalink": "multiple-angels-2-invested-in-isize-technologies-pre-seed--9b03576f--c8bb32d4", "entity_def_id": "investment"}, "investor_stage": ["seed"], "announced_on": "2019-02-04", "organization_identifier": {"uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "funding_round_identifier": {"uuid": "9b03576f-698a-4e94-a4ee-7f29ddb63456", "value": "Pre Seed Round - iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies-pre-seed--9b03576f", "entity_def_id": "funding_round"}, "funding_round_money_raised": {"value": 200000, "currency": "GBP", "value_usd": 260543}, "funding_round_investment_type": "pre_seed", "investor_type": ["investment_partner", "angel"], "created_at": "2020-05-13T05:51:26Z", "updated_at": "2020-05-13T05:51:26Z", "investor_identifier": {"uuid": "1e5e631a-345c-ad12-b301-0e0a0b86f271", "value": "Multiple Angels", "permalink": "multiple-angels-2", "entity_def_id": "person"}}, {"identifier": {"uuid": "381d97b8-c723-4564-a690-c56152651585", "value": "Multiple Angels investment in Pre Seed Round - iSIZE Technologies", "permalink": "multiple-angels-2-invested-in-isize-technologies-pre-seed--9766f01c--381d97b8", "entity_def_id": "investment"}, "investor_stage": ["seed"], "announced_on": "2018-03-01", "organization_identifier": {"uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "funding_round_identifier": {"uuid": "9766f01c-6b7c-4cd9-bf19-8a29e64b8806", "value": "Pre Seed Round - iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies-pre-seed--9766f01c", "entity_def_id": "funding_round"}, "funding_round_money_raised": {"value": 375000, "currency": "GBP", "value_usd": 516570}, "funding_round_investment_type": "pre_seed", "investor_type": ["investment_partner", "angel"], "created_at": "2020-05-13T05:56:59Z", "updated_at": "2020-05-13T05:56:59Z", "investor_identifier": {"uuid": "1e5e631a-345c-ad12-b301-0e0a0b86f271", "value": "Multiple Angels", "permalink": "multiple-angels-2", "entity_def_id": "person"}}], "participated_funds": [], "raised_funding_rounds": [{"num_partners": 1, "identifier": {"uuid": "b5bba780-737a-492a-98e3-9298e2a4266e", "value": "Series A - iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies-series-a--b5bba780", "entity_def_id": "funding_round"}, "rank_funding_round": 444253, "announced_on": "2021-05-06", "money_raised": {"value": 4500000, "currency": "GBP", "value_usd": 6252753}, "short_description": "iSIZE Technologies raised $6252752 on 2021-05-06 in Series A", "funded_organization_funding_stage": "early_stage_venture", "investment_stage": "early_stage_venture", "num_investors": 2, "is_equity": true, "funded_organization_categories": [{"uuid": "c4d8caf3-5fe7-359b-f9f2-2d708378e4ee", "value": "Artificial Intelligence", "permalink": "artificial-intelligence", "entity_def_id": "category"}, {"uuid": "dbca89fa-f083-5438-b4ad-d3fdeceb78e7", "value": "Information Technology", "permalink": "information-technology-dbca", "entity_def_id": "category"}, {"uuid": "5ea0cdb7-c9a6-47fc-50f8-c9b0fac04863", "value": "Machine Learning", "permalink": "machine-learning", "entity_def_id": "category"}, {"uuid": "c08b5441-a05b-9777-b7a6-012728caddd9", "value": "Software", "permalink": "software", "entity_def_id": "category"}, {"uuid": "c6adf822-7fa3-c0b3-5188-556e08d72128", "value": "Video", "permalink": "video", "entity_def_id": "category"}, {"uuid": "e5445f23-204d-6d79-5309-2c17f1e66d78", "value": "Video on Demand", "permalink": "video-on-demand-e544", "entity_def_id": "category"}, {"uuid": "b1b3b2d7-85ed-2cb1-fc60-3e2b6a3b5ddd", "value": "Video Streaming", "permalink": "video-streaming", "entity_def_id": "category"}], "funded_organization_location": [{"uuid": "aad17950-576b-8c44-8fd4-f44dbeb59220", "value": "London", "permalink": "london-england", "entity_def_id": "location", "location_type": "city"}, {"uuid": "79eb923b-9e93-e0db-2fe0-75f0c430c2cb", "value": "England", "permalink": "england-united-kingdom", "entity_def_id": "location", "location_type": "region"}, {"uuid": "a30e342c-1742-6b1c-66e9-461de680e54b", "value": "United Kingdom", "permalink": "united-kingdom", "entity_def_id": "location", "location_type": "country"}, {"uuid": "6106f5dc-823e-5da8-40d7-51612c0b2c4e", "value": "Europe", "permalink": "europe", "entity_def_id": "location", "location_type": "continent"}], "funded_organization_funding_total": {"value": 5815000, "currency": "GBP", "value_usd": 7942415}, "investment_type": "series_a", "lead_investor_identifiers": [{"uuid": "c901d65e-c346-e1a8-7a87-58dd6ca991f9", "value": "Octopus Ventures", "image_id": "kl5vvz6vbs6qzn32fkyz", "permalink": "octopus-ventures", "entity_def_id": "organization"}], "investor_identifiers": [{"role": "investor", "uuid": "c901d65e-c346-e1a8-7a87-58dd6ca991f9", "value": "Octopus Ventures", "image_id": "kl5vvz6vbs6qzn32fkyz", "permalink": "octopus-ventures", "entity_def_id": "organization"}, {"role": "investor", "uuid": "0cf953b7-00bb-e607-e775-f781b1b7cf51", "value": "TD Veen", "image_id": "v1431928192/dn2s3anncilsctiyqsxk.png", "permalink": "td-veen", "entity_def_id": "organization"}], "funded_organization_identifier": {"role": "investee", "uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "created_at": "2021-05-06T19:40:03Z", "funded_organization_description": "iSIZE Technologies is a company to delivers deep neural networks for data-driven video preprocessing.", "updated_at": "2021-05-06T20:02:22Z"}, {"identifier": {"uuid": "423015a6-011f-4e54-8cbc-c9fe0c2e2298", "value": "Seed Round - iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies-seed--423015a6", "entity_def_id": "funding_round"}, "rank_funding_round": 176509, "announced_on": "2020-05-11", "money_raised": {"value": 740000, "currency": "GBP", "value_usd": 912549}, "short_description": "iSIZE Technologies raised $912549 on 2020-05-11 in Seed Round", "funded_organization_funding_stage": "early_stage_venture", "investment_stage": "seed", "num_investors": 6, "is_equity": true, "funded_organization_categories": [{"uuid": "c4d8caf3-5fe7-359b-f9f2-2d708378e4ee", "value": "Artificial Intelligence", "permalink": "artificial-intelligence", "entity_def_id": "category"}, {"uuid": "dbca89fa-f083-5438-b4ad-d3fdeceb78e7", "value": "Information Technology", "permalink": "information-technology-dbca", "entity_def_id": "category"}, {"uuid": "5ea0cdb7-c9a6-47fc-50f8-c9b0fac04863", "value": "Machine Learning", "permalink": "machine-learning", "entity_def_id": "category"}, {"uuid": "c08b5441-a05b-9777-b7a6-012728caddd9", "value": "Software", "permalink": "software", "entity_def_id": "category"}, {"uuid": "c6adf822-7fa3-c0b3-5188-556e08d72128", "value": "Video", "permalink": "video", "entity_def_id": "category"}, {"uuid": "e5445f23-204d-6d79-5309-2c17f1e66d78", "value": "Video on Demand", "permalink": "video-on-demand-e544", "entity_def_id": "category"}, {"uuid": "b1b3b2d7-85ed-2cb1-fc60-3e2b6a3b5ddd", "value": "Video Streaming", "permalink": "video-streaming", "entity_def_id": "category"}], "closed_on": {"value": "2020-05-07", "precision": "day"}, "funded_organization_location": [{"uuid": "aad17950-576b-8c44-8fd4-f44dbeb59220", "value": "London", "permalink": "london-england", "entity_def_id": "location", "location_type": "city"}, {"uuid": "79eb923b-9e93-e0db-2fe0-75f0c430c2cb", "value": "England", "permalink": "england-united-kingdom", "entity_def_id": "location", "location_type": "region"}, {"uuid": "a30e342c-1742-6b1c-66e9-461de680e54b", "value": "United Kingdom", "permalink": "united-kingdom", "entity_def_id": "location", "location_type": "country"}, {"uuid": "6106f5dc-823e-5da8-40d7-51612c0b2c4e", "value": "Europe", "permalink": "europe", "entity_def_id": "location", "location_type": "continent"}], "funded_organization_funding_total": {"value": 5815000, "currency": "GBP", "value_usd": 7942415}, "investment_type": "seed", "lead_investor_identifiers": [{"uuid": "0cf953b7-00bb-e607-e775-f781b1b7cf51", "value": "TD Veen", "image_id": "v1431928192/dn2s3anncilsctiyqsxk.png", "permalink": "td-veen", "entity_def_id": "organization"}], "investor_identifiers": [{"role": "investor", "uuid": "ba2ab90f-1b3a-d5e4-8fc4-8953c9f6313a", "value": "Charles Conn", "image_id": "v1456479977/i0ik5tsrepm6jk9k8aic.jpg", "permalink": "charles-conn", "entity_def_id": "person"}, {"role": "investor", "uuid": "8d851678-9164-e490-a3ae-7abcdb3f2287", "value": "Patrick Pichette", "image_id": "edngsa68mbsaf5or6j9b", "permalink": "patrick-pichette", "entity_def_id": "person"}, {"role": "investor", "uuid": "95a1f09e-6b4f-4aa9-8066-2215c949aaab", "value": "Paul Massara", "image_id": "zefgqo0adxsaeq044bkw", "permalink": "paul-massara-aaab", "entity_def_id": "person"}, {"role": "investor", "uuid": "69294b81-b677-7b37-975a-9640bc1d938a", "value": "Robert Sansom", "image_id": "v1469005050/a7xv7rcazqrtpnirtnlh.png", "permalink": "robert-sansom", "entity_def_id": "person"}, {"role": "investor", "uuid": "0cf953b7-00bb-e607-e775-f781b1b7cf51", "value": "TD Veen", "image_id": "v1431928192/dn2s3anncilsctiyqsxk.png", "permalink": "td-veen", "entity_def_id": "organization"}, {"role": "investor", "uuid": "2d90af63-9abb-fa96-03cd-a5c7e897579c", "value": "William Tunstall-Pedoe", "image_id": "kuhdy2jn93u8i5zmngd5", "permalink": "william-tunstall-pedoe", "entity_def_id": "person"}], "funded_organization_identifier": {"role": "investee", "uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "created_at": "2020-05-13T05:32:29Z", "target_money_raised": {"value": 500000, "currency": "GBP", "value_usd": 616587}, "funded_organization_description": "iSIZE Technologies is a company to delivers deep neural networks for data-driven video preprocessing.", "updated_at": "2020-05-13T05:32:29Z"}, {"identifier": {"uuid": "9b03576f-698a-4e94-a4ee-7f29ddb63456", "value": "Pre Seed Round - iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies-pre-seed--9b03576f", "entity_def_id": "funding_round"}, "rank_funding_round": 247644, "announced_on": "2019-02-04", "money_raised": {"value": 200000, "currency": "GBP", "value_usd": 260543}, "short_description": "iSIZE Technologies raised $260542 on 2019-02-04 in Pre Seed Round", "funded_organization_funding_stage": "early_stage_venture", "investment_stage": "seed", "num_investors": 2, "is_equity": true, "funded_organization_categories": [{"uuid": "c4d8caf3-5fe7-359b-f9f2-2d708378e4ee", "value": "Artificial Intelligence", "permalink": "artificial-intelligence", "entity_def_id": "category"}, {"uuid": "dbca89fa-f083-5438-b4ad-d3fdeceb78e7", "value": "Information Technology", "permalink": "information-technology-dbca", "entity_def_id": "category"}, {"uuid": "5ea0cdb7-c9a6-47fc-50f8-c9b0fac04863", "value": "Machine Learning", "permalink": "machine-learning", "entity_def_id": "category"}, {"uuid": "c08b5441-a05b-9777-b7a6-012728caddd9", "value": "Software", "permalink": "software", "entity_def_id": "category"}, {"uuid": "c6adf822-7fa3-c0b3-5188-556e08d72128", "value": "Video", "permalink": "video", "entity_def_id": "category"}, {"uuid": "e5445f23-204d-6d79-5309-2c17f1e66d78", "value": "Video on Demand", "permalink": "video-on-demand-e544", "entity_def_id": "category"}, {"uuid": "b1b3b2d7-85ed-2cb1-fc60-3e2b6a3b5ddd", "value": "Video Streaming", "permalink": "video-streaming", "entity_def_id": "category"}], "closed_on": {"value": "2019-02-03", "precision": "day"}, "funded_organization_location": [{"uuid": "aad17950-576b-8c44-8fd4-f44dbeb59220", "value": "London", "permalink": "london-england", "entity_def_id": "location", "location_type": "city"}, {"uuid": "79eb923b-9e93-e0db-2fe0-75f0c430c2cb", "value": "England", "permalink": "england-united-kingdom", "entity_def_id": "location", "location_type": "region"}, {"uuid": "a30e342c-1742-6b1c-66e9-461de680e54b", "value": "United Kingdom", "permalink": "united-kingdom", "entity_def_id": "location", "location_type": "country"}, {"uuid": "6106f5dc-823e-5da8-40d7-51612c0b2c4e", "value": "Europe", "permalink": "europe", "entity_def_id": "location", "location_type": "continent"}], "funded_organization_funding_total": {"value": 5815000, "currency": "GBP", "value_usd": 7942415}, "investment_type": "pre_seed", "investor_identifiers": [{"role": "investor", "uuid": "248e28ee-db49-a613-6a5e-6120f68456ad", "value": "Creative Destruction Lab (CDL)", "image_id": "v1427491020/lkxian3nnkephyzeagws.jpg", "permalink": "creative-destruction-lab", "entity_def_id": "organization"}, {"role": "investor", "uuid": "1e5e631a-345c-ad12-b301-0e0a0b86f271", "value": "Multiple Angels", "permalink": "multiple-angels-2", "entity_def_id": "person"}], "funded_organization_identifier": {"role": "investee", "uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "created_at": "2020-05-13T05:51:26Z", "funded_organization_description": "iSIZE Technologies is a company to delivers deep neural networks for data-driven video preprocessing.", "updated_at": "2020-05-13T05:51:26Z"}, {"identifier": {"uuid": "9766f01c-6b7c-4cd9-bf19-8a29e64b8806", "value": "Pre Seed Round - iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies-pre-seed--9766f01c", "entity_def_id": "funding_round"}, "rank_funding_round": 240051, "announced_on": "2018-03-01", "money_raised": {"value": 375000, "currency": "GBP", "value_usd": 516570}, "short_description": "iSIZE Technologies raised $516570 on 2018-03-01 in Pre Seed Round", "funded_organization_funding_stage": "early_stage_venture", "investment_stage": "seed", "num_investors": 1, "is_equity": true, "funded_organization_categories": [{"uuid": "c4d8caf3-5fe7-359b-f9f2-2d708378e4ee", "value": "Artificial Intelligence", "permalink": "artificial-intelligence", "entity_def_id": "category"}, {"uuid": "dbca89fa-f083-5438-b4ad-d3fdeceb78e7", "value": "Information Technology", "permalink": "information-technology-dbca", "entity_def_id": "category"}, {"uuid": "5ea0cdb7-c9a6-47fc-50f8-c9b0fac04863", "value": "Machine Learning", "permalink": "machine-learning", "entity_def_id": "category"}, {"uuid": "c08b5441-a05b-9777-b7a6-012728caddd9", "value": "Software", "permalink": "software", "entity_def_id": "category"}, {"uuid": "c6adf822-7fa3-c0b3-5188-556e08d72128", "value": "Video", "permalink": "video", "entity_def_id": "category"}, {"uuid": "e5445f23-204d-6d79-5309-2c17f1e66d78", "value": "Video on Demand", "permalink": "video-on-demand-e544", "entity_def_id": "category"}, {"uuid": "b1b3b2d7-85ed-2cb1-fc60-3e2b6a3b5ddd", "value": "Video Streaming", "permalink": "video-streaming", "entity_def_id": "category"}], "funded_organization_location": [{"uuid": "aad17950-576b-8c44-8fd4-f44dbeb59220", "value": "London", "permalink": "london-england", "entity_def_id": "location", "location_type": "city"}, {"uuid": "79eb923b-9e93-e0db-2fe0-75f0c430c2cb", "value": "England", "permalink": "england-united-kingdom", "entity_def_id": "location", "location_type": "region"}, {"uuid": "a30e342c-1742-6b1c-66e9-461de680e54b", "value": "United Kingdom", "permalink": "united-kingdom", "entity_def_id": "location", "location_type": "country"}, {"uuid": "6106f5dc-823e-5da8-40d7-51612c0b2c4e", "value": "Europe", "permalink": "europe", "entity_def_id": "location", "location_type": "continent"}], "funded_organization_funding_total": {"value": 5815000, "currency": "GBP", "value_usd": 7942415}, "investment_type": "pre_seed", "investor_identifiers": [{"role": "investor", "uuid": "1e5e631a-345c-ad12-b301-0e0a0b86f271", "value": "Multiple Angels", "permalink": "multiple-angels-2", "entity_def_id": "person"}], "funded_organization_identifier": {"role": "investee", "uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "created_at": "2020-05-13T05:56:59Z", "funded_organization_description": "iSIZE Technologies is a company to delivers deep neural networks for data-driven video preprocessing.", "updated_at": "2020-05-13T05:56:59Z"}], "child_ownerships": [], "founders": [{"first_name": "Sergio", "rank_delta_d30": 0.3, "image_url": "https://res.cloudinary.com/crunchbase-production/image/upload/tbybdokidnthuq2e9u5l", "website": {"value": "https://www.isize.co/"}, "num_founded_organizations": 1, "identifier": {"uuid": "3730d28c-bbad-4f0a-9144-807a73ccc753", "value": "Sergio Grce", "image_id": "tbybdokidnthuq2e9u5l", "permalink": "sergio-grce", "entity_def_id": "person"}, "primary_job_title": "Founder & CEO", "description": "Sergio Grce holds an MSc in Investment Management and a First Class Hons BSc in Economics. During his 12-year investment banking career, Sergio worked for some of the largest investment banks, including Merrill Lynch, Nomura, Royal Bank of Scotland and Jeffries. In 2016, through Sergio\u2019s strong interest in AI technologies, as well as its applications for video imaging, art and gaming, iSIZE was formed utilising Sergio\u2019s leadership, ability in bringing a strong network and teams together, leading both technical and commercial operations.", "linkedin": {"value": "https://www.linkedin.com/in/sergio-grce-9753441/"}, "short_description": "Sergio Grce holds an MSc in Investment Management and a First Class Hons BSc in Economics. During his 12-year investment banking career, Ser...", "num_jobs": 1, "facebook": {"value": "https://www.facebook.com/isize.co/"}, "last_name": "Grce", "primary_organization": {"uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "rank_person": 203038, "created_at": "2019-06-03T07:17:06Z", "num_articles": 8, "num_current_jobs": 1, "twitter": {"value": "https://twitter.com/isize_co"}, "rank_delta_d90": -1.1, "website_url": "https://www.isize.co/", "updated_at": "2022-02-26T06:47:38Z", "rank_delta_d7": 0.8, "gender": "male"}, {"aliases": ["Yiannis"], "first_name": "Yiannis", "rank_delta_d30": 0.4, "image_url": "https://res.cloudinary.com/crunchbase-production/image/upload/kfqafjauihvf5qpwnim3", "website": {"value": "https://www.isize.co/"}, "num_founded_organizations": 1, "identifier": {"uuid": "c153e92d-1e37-4bb4-aabd-7ff10862f97e", "value": "Yiannis Andreopoulos", "image_id": "kfqafjauihvf5qpwnim3", "permalink": "yiannis-andreopoulos", "entity_def_id": "person"}, "primary_job_title": "Founder & Chief Technology Officer", "description": "Yiannis Andreopoulos has 20 years\u2019 experience in video coding and machine learning, with 150+ research papers, several patent applications, 10 contributions to JPEG/MPEG standards and extensive engagement with industry in consulting and advisory roles. As CTO of iSIZE Technologies, he is dedicated to challenging the video delivery industry to see how deep neural networks can enhance and accelerate existing video coding standards for internet video.", "linkedin": {"value": "https://www.linkedin.com/in/yiannis-andreopoulos-1758bb9b"}, "short_description": "Yiannis Andreopoulos has 20 years\u2019 experience in video coding and machine learning, with 150+ research papers, several patent applications, ...", "num_jobs": 1, "facebook": {"value": "https://www.facebook.com/isize.co/"}, "last_name": "Andreopoulos", "primary_organization": {"uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "rank_person": 868707, "created_at": "2019-07-25T16:39:27Z", "num_articles": 13, "num_current_jobs": 1, "num_event_appearances": 7, "twitter": {"value": "https://twitter.com/isize_co"}, "rank_delta_d90": -0.1, "website_url": "https://www.isize.co/", "updated_at": "2022-04-08T19:44:45Z", "rank_delta_d7": 0.1, "gender": "male"}], "ipos": [], "raised_funds": [], "parent_organization": [], "acquirer_acquisitions": [], "parent_ownership": [], "fields": {"company_type": "for_profit", "aliases": ["iSIZE"], "rank_delta_d30": -0.5, "image_url": "https://res.cloudinary.com/crunchbase-production/image/upload/xmuwkvi2citoghycs1tu", "founded_on": {"value": "2016-09-22", "precision": "day"}, "website": {"value": "https://www.isize.co"}, "equity_funding_total": {"value": 5815000, "currency": "GBP", "value_usd": 7942415}, "identifier": {"uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "ipo_status": "private", "founder_identifiers": [{"uuid": "3730d28c-bbad-4f0a-9144-807a73ccc753", "value": "Sergio Grce", "image_id": "tbybdokidnthuq2e9u5l", "permalink": "sergio-grce", "entity_def_id": "person"}, {"uuid": "c153e92d-1e37-4bb4-aabd-7ff10862f97e", "value": "Yiannis Andreopoulos", "image_id": "kfqafjauihvf5qpwnim3", "permalink": "yiannis-andreopoulos", "entity_def_id": "person"}], "description": "iSIZE Technologies is a company to delivers deep neural networks for data-driven video preprocessing in order to provide significant bitrate savings over any standard encoder, or significant quality improvement at the same bitrate. This will also allow for other advanced ai services like automated tagging and recognition to run seamlessly in conjunction with live content delivery.", "category_groups": [{"uuid": "e5514a50-8200-7f6b-de87-b07990670800", "value": "Artificial Intelligence", "permalink": "artificial-intelligence-e551", "entity_def_id": "category_group"}, {"uuid": "d5f76b18-83d5-c86d-25cb-facca572d71a", "value": "Content and Publishing", "permalink": "content-and-publishing", "entity_def_id": "category_group"}, {"uuid": "701eef4f-18c1-4aff-b550-caf732cd575f", "value": "Data and Analytics", "permalink": "data-and-analytics", "entity_def_id": "category_group"}, {"uuid": "4fe3f3ac-e522-5889-7477-c1b6d6663710", "value": "Information Technology", "permalink": "information-technology-4fe3", "entity_def_id": "category_group"}, {"uuid": "3bd75a3d-986a-9911-b145-f944d7d6d4c6", "value": "Media and Entertainment", "permalink": "media-and-entertainment-3bd7", "entity_def_id": "category_group"}, {"uuid": "285e29fc-8f70-bf00-1749-9e94158f64f4", "value": "Science and Engineering", "permalink": "science-and-engineering", "entity_def_id": "category_group"}, {"uuid": "85b6bca9-930a-11bc-a608-a513b76fb637", "value": "Software", "permalink": "software-85b6", "entity_def_id": "category_group"}, {"uuid": "d4574740-e74d-5087-fc0f-8244b32d6b99", "value": "Video", "permalink": "video-d457", "entity_def_id": "category_group"}], "linkedin": {"value": "https://www.linkedin.com/company/isize.co/"}, "short_description": "iSIZE Technologies is a company to delivers deep neural networks for data-driven video preprocessing.", "num_current_positions": 3, "num_investors": 9, "operating_status": "active", "num_lead_investors": 2, "rank_org": 37267, "facebook": {"value": "https://en-gb.facebook.com/isize.co/"}, "num_employees_enum": "c_00011_00050", "status": "operating", "funding_total": {"value": 5815000, "currency": "GBP", "value_usd": 7942415}, "num_funding_rounds": 4, "last_equity_funding_type": "series_a", "investor_identifiers": [{"uuid": "248e28ee-db49-a613-6a5e-6120f68456ad", "value": "Creative Destruction Lab (CDL)", "image_id": "v1427491020/lkxian3nnkephyzeagws.jpg", "permalink": "creative-destruction-lab", "entity_def_id": "organization"}, {"uuid": "c901d65e-c346-e1a8-7a87-58dd6ca991f9", "value": "Octopus Ventures", "image_id": "kl5vvz6vbs6qzn32fkyz", "permalink": "octopus-ventures", "entity_def_id": "organization"}, {"uuid": "2d90af63-9abb-fa96-03cd-a5c7e897579c", "value": "William Tunstall-Pedoe", "image_id": "kuhdy2jn93u8i5zmngd5", "permalink": "william-tunstall-pedoe", "entity_def_id": "person"}, {"uuid": "8d851678-9164-e490-a3ae-7abcdb3f2287", "value": "Patrick Pichette", "image_id": "edngsa68mbsaf5or6j9b", "permalink": "patrick-pichette", "entity_def_id": "person"}, {"uuid": "0cf953b7-00bb-e607-e775-f781b1b7cf51", "value": "TD Veen", "image_id": "v1431928192/dn2s3anncilsctiyqsxk.png", "permalink": "td-veen", "entity_def_id": "organization"}], "last_funding_type": "series_a", "categories": [{"uuid": "c4d8caf3-5fe7-359b-f9f2-2d708378e4ee", "value": "Artificial Intelligence", "permalink": "artificial-intelligence", "entity_def_id": "category"}, {"uuid": "dbca89fa-f083-5438-b4ad-d3fdeceb78e7", "value": "Information Technology", "permalink": "information-technology-dbca", "entity_def_id": "category"}, {"uuid": "5ea0cdb7-c9a6-47fc-50f8-c9b0fac04863", "value": "Machine Learning", "permalink": "machine-learning", "entity_def_id": "category"}, {"uuid": "c08b5441-a05b-9777-b7a6-012728caddd9", "value": "Software", "permalink": "software", "entity_def_id": "category"}, {"uuid": "c6adf822-7fa3-c0b3-5188-556e08d72128", "value": "Video", "permalink": "video", "entity_def_id": "category"}, {"uuid": "e5445f23-204d-6d79-5309-2c17f1e66d78", "value": "Video on Demand", "permalink": "video-on-demand-e544", "entity_def_id": "category"}, {"uuid": "b1b3b2d7-85ed-2cb1-fc60-3e2b6a3b5ddd", "value": "Video Streaming", "permalink": "video-streaming", "entity_def_id": "category"}], "legal_name": "iSIZE Limited.", "created_at": "2018-08-28T10:26:30Z", "location_identifiers": [{"uuid": "aad17950-576b-8c44-8fd4-f44dbeb59220", "value": "London", "permalink": "london-england", "entity_def_id": "location", "location_type": "city"}, {"uuid": "79eb923b-9e93-e0db-2fe0-75f0c430c2cb", "value": "England", "permalink": "england-united-kingdom", "entity_def_id": "location", "location_type": "region"}, {"uuid": "a30e342c-1742-6b1c-66e9-461de680e54b", "value": "United Kingdom", "permalink": "united-kingdom", "entity_def_id": "location", "location_type": "country"}, {"uuid": "6106f5dc-823e-5da8-40d7-51612c0b2c4e", "value": "Europe", "permalink": "europe", "entity_def_id": "location", "location_type": "continent"}], "num_articles": 119, "last_funding_at": "2021-05-06", "num_event_appearances": 5, "twitter": {"value": "https://twitter.com/isize_co"}, "rank_delta_d90": -3.8, "website_url": "https://www.isize.co", "updated_at": "2022-07-19T01:21:50Z", "last_equity_funding_total": {"value": 4500000.0, "currency": "GBP", "value_usd": 6252752.948173015}, "contact_email": "info@isize.co", "funding_stage": "early_stage_venture", "rank_delta_d7": -0.1, "last_funding_total": {"value": 4500000.0, "currency": "GBP", "value_usd": 6252752.948173015}, "num_founders": 2}, "jobs": [{"is_current": true, "person_identifier": {"uuid": "fd0e0d35-0083-4ba4-8cf1-142a6d5885c5", "value": "Jasna Vicentijevic", "image_id": "spo8ploxkztv0aq8oxqe", "permalink": "jasna-vicentijevic", "entity_def_id": "person"}, "identifier": {"uuid": "5147877b-ea6c-4b77-a47e-d295df63dd91", "value": "Jasna Vicentijevic Chief Operating Officer @ iSIZE Technologies", "permalink": "jasna-vicentijevic-executive-isize-technologies--5147877b", "entity_def_id": "job"}, "short_description": "Jasna Vicentijevic Chief Operating Officer @ iSIZE Technologies", "organization_identifier": {"uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "created_at": "2022-01-24T12:46:44Z", "title": "Chief Operating Officer", "updated_at": "2022-01-24T12:47:35Z", "job_type": "executive"}, {"is_current": true, "person_identifier": {"uuid": "c153e92d-1e37-4bb4-aabd-7ff10862f97e", "value": "Yiannis Andreopoulos", "image_id": "kfqafjauihvf5qpwnim3", "permalink": "yiannis-andreopoulos", "entity_def_id": "person"}, "identifier": {"uuid": "ef000890-a36a-4c20-8436-ef8b53a1fcfd", "value": "Yiannis Andreopoulos Founder & Chief Technology Officer @ iSIZE Technologies", "permalink": "yiannis-andreopoulos-executive-isize-technologies--ef000890", "entity_def_id": "job"}, "short_description": "Yiannis Andreopoulos Founder & Chief Technology Officer @ iSIZE Technologies", "organization_identifier": {"uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "created_at": "2019-07-29T09:37:20Z", "title": "Founder & Chief Technology Officer", "updated_at": "2022-01-26T11:37:35Z", "job_type": "executive"}, {"is_current": true, "person_identifier": {"uuid": "3730d28c-bbad-4f0a-9144-807a73ccc753", "value": "Sergio Grce", "image_id": "tbybdokidnthuq2e9u5l", "permalink": "sergio-grce", "entity_def_id": "person"}, "identifier": {"uuid": "f74af0df-2fd5-42f0-9ad3-8d6427e0a1aa", "value": "Sergio Grce Founder & CEO @ iSIZE Technologies", "permalink": "sergio-grce-executive-isize-technologies--f74af0df", "entity_def_id": "job"}, "short_description": "Sergio Grce Founder & CEO @ iSIZE Technologies", "organization_identifier": {"uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "created_at": "2019-06-03T07:17:06Z", "started_on": {"value": "2016-11-01", "precision": "month"}, "title": "Founder & CEO", "updated_at": "2019-06-03T07:17:06Z", "job_type": "executive"}], "investors": [{"first_name": "Charles", "rank_delta_d30": -0.2, "image_url": "https://res.cloudinary.com/crunchbase-production/image/upload/v1456479977/i0ik5tsrepm6jk9k8aic.jpg", "rank_principal": 2147708, "identifier": {"uuid": "ba2ab90f-1b3a-d5e4-8fc4-8953c9f6313a", "value": "Charles Conn", "image_id": "v1456479977/i0ik5tsrepm6jk9k8aic.jpg", "permalink": "charles-conn", "entity_def_id": "person"}, "description": "Charles Conn previously serves as the CEO at Citysearch. He is a mentor at Creative Destruction Labs.", "investor_stage": ["seed"], "linkedin": {"value": "http://www.linkedin.com/pub/charles-conn/4/890/500"}, "short_description": "Charles Conn previously serves as the CEO at Citysearch. He is a mentor at Creative Destruction Labs.", "num_partner_investments": 2, "num_portfolio_organizations": 2, "num_investments": 2, "num_jobs": 6, "last_name": "Conn", "investor_type": ["investment_partner", "angel"], "created_at": "2011-09-17T13:59:34Z", "location_identifiers": [{"uuid": "c1af6713-07f3-7fef-0b8c-8ae1d7e71bb3", "value": "Oxford", "permalink": "oxford-oxfordshire", "entity_def_id": "location", "location_type": "city"}, {"uuid": "517ebbb4-6a4d-3a8a-41de-b8af68d48c69", "value": "Oxfordshire", "permalink": "oxfordshire-united-kingdom", "entity_def_id": "location", "location_type": "region"}, {"uuid": "a30e342c-1742-6b1c-66e9-461de680e54b", "value": "United Kingdom", "permalink": "united-kingdom", "entity_def_id": "location", "location_type": "country"}, {"uuid": "6106f5dc-823e-5da8-40d7-51612c0b2c4e", "value": "Europe", "permalink": "europe", "entity_def_id": "location", "location_type": "continent"}], "num_articles": 3, "rank_delta_d90": -1, "updated_at": "2021-06-16T05:03:06Z", "rank_delta_d7": 0, "gender": "male"}, {"company_type": "non_profit", "location_group_identifiers": [{"uuid": "645645b9-0033-4c72-aa83-2ae8c6062d55", "value": "Great Lakes", "permalink": "great-lakes-north-america", "entity_def_id": "location", "location_type": "group"}], "aliases": ["CDL"], "rank_delta_d30": 5.2, "image_url": "https://res.cloudinary.com/crunchbase-production/image/upload/v1427491020/lkxian3nnkephyzeagws.jpg", "founded_on": {"value": "2012-08-01", "precision": "day"}, "website": {"value": "http://www.creativedestructionlab.com/"}, "num_lead_investments": 190, "equity_funding_total": {"value": 5600000, "currency": "USD", "value_usd": 5600000}, "rank_principal": 1697, "num_funds": 1, "program_application_deadline": "2021-07-30", "num_diversity_spotlight_investments": 37, "identifier": {"uuid": "248e28ee-db49-a613-6a5e-6120f68456ad", "value": "Creative Destruction Lab (CDL)", "image_id": "v1427491020/lkxian3nnkephyzeagws.jpg", "permalink": "creative-destruction-lab", "entity_def_id": "organization"}, "ipo_status": "private", "founder_identifiers": [{"uuid": "06d53c3b-831b-4205-92ec-0b8466e9c445", "value": "Ajay Agrawal", "image_id": "aaychw8iiwwg3wti47le", "permalink": "ajay-agrawal-c445", "entity_def_id": "person"}, {"uuid": "f2473dc2-0e6c-4a7d-a0a6-31ff0f9f5d18", "value": "Amir Sariri", "image_id": "itqib3n9s3kgaohrbprw", "permalink": "amir-sariri-5d18", "entity_def_id": "person"}, {"uuid": "d8a0475e-cac3-bd1c-91f0-179d2e37e808", "value": "Anthony Lacavera", "image_id": "v1397186183/0075092a92c4515da6d2b2e264b12773.png", "permalink": "anthony-lacavera", "entity_def_id": "person"}], "description": "Creative Destruction Lab (CDL) is a no-fee/no-equity organization that delivers an objectives-based mentoring program for massively scalable, pre-seed stage, science- and technology-based companies. Its nine-month program allows founders to learn from experienced entrepreneurs, increasing their likelihood of success. Founded in 2012 by Professor Ajay Agrawal at the University of Toronto\u2019s Rotman School of Management, the program has expanded to nine sites across four countries: Oxford (Sa\u00efd Business School, University of Oxford), Paris (HEC Paris), Atlanta (Scheller College of Business, Georgia Institute of Technology), Madison (Wisconsin School of Business, University of Wisconsin-Madison), Vancouver (Sauder School of Business, University of British Columbia), Montreal (HEC Montr\u00e9al), Calgary (Haskayne School of Business, University of Calgary), and Halifax (Rowe School of Business, Dalhousie University).\n\nDue to the current global COVID-19 crisis, we have adapted our program to be delivered entirely online. As we follow the guidance of public health officials, we will re-evaluate periodically to decide when to transition back to in-person format. CDL will not turn away ventures that have applied to a specific location if they are not able to attend a session in-person, as there is still uncertainty around the COVID-19 situation.", "investor_stage": ["early_stage_venture", "seed"], "category_groups": [{"uuid": "3805abe5-5b70-a4a1-a51e-bcda8257aca0", "value": "Commerce and Shopping", "permalink": "commerce-and-shopping", "entity_def_id": "category_group"}, {"uuid": "2e6eafef-f310-ba60-d932-62f866a87779", "value": "Hardware", "permalink": "hardware-2e6e", "entity_def_id": "category_group"}, {"uuid": "133d294c-e5d0-2c4f-9acc-aed0ada1fa8a", "value": "Manufacturing", "permalink": "manufacturing-133d", "entity_def_id": "category_group"}, {"uuid": "1718a583-5105-47e5-b3b0-16c2e3dca322", "value": "Other", "entity_def_id": "category_group"}, {"uuid": "abe5fae0-90ed-1f76-2bfa-dc5ee53af3b5", "value": "Professional Services", "permalink": "professional-services-abe5", "entity_def_id": "category_group"}, {"uuid": "285e29fc-8f70-bf00-1749-9e94158f64f4", "value": "Science and Engineering", "permalink": "science-and-engineering", "entity_def_id": "category_group"}, {"uuid": "85b6bca9-930a-11bc-a608-a513b76fb637", "value": "Software", "permalink": "software-85b6", "entity_def_id": "category_group"}, {"uuid": "5142a167-54a3-e7af-765b-e5de07063dd3", "value": "Sustainability", "permalink": "sustainability-5142", "entity_def_id": "category_group"}, {"uuid": "adc31356-a675-00a1-305e-8becd771319e", "value": "Transportation", "permalink": "transportation-adc3", "entity_def_id": "category_group"}], "linkedin": {"value": "https://www.linkedin.com/school/creative-destruction-lab/"}, "program_duration": 40, "short_description": "Creative Destruction Lab (CDL) is a no-fee/no-equity mentoring program for the founders and executives of pre-seed stage ventures.", "num_portfolio_organizations": 754, "num_investments": 763, "revenue_range": "r_00001000", "num_investors": 6, "operating_status": "active", "num_lead_investors": 6, "num_exits_ipo": 51, "facebook": {"value": "https://www.facebook.com/creativedestructionlab"}, "num_employees_enum": "c_00101_00250", "status": "operating", "funding_total": {"value": 31497662, "currency": "USD", "value_usd": 31497662}, "num_funding_rounds": 5, "last_equity_funding_type": "series_unknown", "hub_tags": ["cbvp"], "program_type": "on_site", "investor_identifiers": [{"uuid": "7d57a47c-b60a-b14a-9854-9edc54556430", "value": "Bank of Montreal", "image_id": "ks3axahixcjlhbm1gcf4", "permalink": "bank-of-montreal", "entity_def_id": "organization"}, {"uuid": "cb5bb447-6306-4306-a50e-cdace223fd62", "value": "SCALE AI", "image_id": "xeu38fb8jcihzhf7fbej", "permalink": "scale-ai", "entity_def_id": "organization"}, {"uuid": "9076754e-e6cb-6b3a-3a97-eec77a1b822e", "value": "RBC Capital", "image_id": "v1397760804/0c8f2589508a0ac71a45b3f4d9937382.png", "permalink": "royal-bank-of-canada", "entity_def_id": "organization"}, {"uuid": "aee47d3b-e06a-d516-6684-1d755ecbe99a", "value": "Government of Canada", "image_id": "v1425296784/ybkasvrljenfltk72cae.png", "permalink": "government-of-canada", "entity_def_id": "organization"}, {"uuid": "45ce21a0-0d1b-1b56-6794-1c3e12f2afd6", "value": "Scotiabank", "image_id": "urzxrbb0fxhlqld2wwss", "permalink": "scotiabank", "entity_def_id": "organization"}], "last_funding_type": "series_unknown", "categories": [{"uuid": "3082b9fa-e0fd-150a-1f2d-a6f8d3515c0c", "value": "Advanced Materials", "permalink": "advanced-materials", "entity_def_id": "category"}, {"uuid": "8a20023f-3e32-20bb-b6ab-e824a0386367", "value": "Aerospace", "permalink": "aerospace-8a20", "entity_def_id": "category"}, {"uuid": "1fea6201-3b7c-4c91-96ec-543f5eded46a", "value": "Blockchain", "permalink": "blockchain", "entity_def_id": "category"}, {"uuid": "99a87fee-dfc7-e887-2e41-5bdf00577cbb", "value": "CleanTech", "permalink": "cleantech", "entity_def_id": "category"}, {"uuid": "93872d37-a1f6-a0af-b9e2-0b150c79f8eb", "value": "Retail Technology", "permalink": "retail-technology", "entity_def_id": "category"}, {"uuid": "99763baa-917d-488e-86e2-34bbbde18450", "value": "Risk Management", "permalink": "risk-management", "entity_def_id": "category"}, {"uuid": "7ec4e4c8-a648-da5d-c02b-90a98c5ebfcf", "value": "Supply Chain Management", "permalink": "supply-chain-management", "entity_def_id": "category"}], "investor_type": ["entrepreneurship_program", "university_program"], "created_at": "2015-03-27T21:16:32Z", "location_identifiers": [{"uuid": "064d4c93-264e-92b0-f3cb-a9ad94cacd77", "value": "Toronto", "permalink": "toronto-ontario", "entity_def_id": "location", "location_type": "city"}, {"uuid": "452160a8-eaea-9fb8-11fe-78c5935ec12e", "value": "Ontario", "permalink": "ontario-canada", "entity_def_id": "location", "location_type": "region"}, {"uuid": "1b3d6217-ec76-1b13-c47a-aca49b466a2c", "value": "Canada", "permalink": "canada", "entity_def_id": "location", "location_type": "country"}], "num_articles": 76, "last_funding_at": "2022-10-13", "num_event_appearances": 3, "twitter": {"value": "https://twitter.com/creativedlab"}, "rank_delta_d90": -0.2, "website_url": "http://www.creativedestructionlab.com/", "updated_at": "2022-01-27T17:47:34Z", "last_equity_funding_total": {"value": 1600000, "currency": "USD", "value_usd": 1600000}, "contact_email": "info@creativedestructionlab.com", "num_exits": 51, "rank_delta_d7": 1.3, "last_funding_total": {"value": 1600000, "currency": "USD", "value_usd": 1600000}, "num_founders": 3}, {"first_name": "Multiple", "rank_delta_d30": -0.5, "rank_principal": 1966761, "num_diversity_spotlight_investments": 4, "identifier": {"uuid": "1e5e631a-345c-ad12-b301-0e0a0b86f271", "value": "Multiple Angels", "permalink": "multiple-angels-2", "entity_def_id": "person"}, "investor_stage": ["seed"], "short_description": "Multiple Angels", "num_partner_investments": 1, "num_portfolio_organizations": 8, "num_investments": 9, "num_exits_ipo": 1, "last_name": "Angels", "investor_type": ["investment_partner", "angel"], "created_at": "2017-10-02T23:19:04Z", "rank_delta_d90": 5.4, "updated_at": "2018-02-12T23:45:14Z", "num_exits": 1, "rank_delta_d7": -0.2, "gender": "not_provided"}, {"aliases": ["Octopus Ventures"], "rank_delta_d30": -0.6, "image_url": "https://res.cloudinary.com/crunchbase-production/image/upload/kl5vvz6vbs6qzn32fkyz", "founded_on": {"value": "2008-01-01", "precision": "year"}, "website": {"value": "http://www.octopusventures.com"}, "num_lead_investments": 214, "rank_principal": 174038, "num_funds": 6, "phone_number": "+44 20 7776 7968", "num_diversity_spotlight_investments": 1, "identifier": {"uuid": "c901d65e-c346-e1a8-7a87-58dd6ca991f9", "value": "Octopus Ventures", "image_id": "kl5vvz6vbs6qzn32fkyz", "permalink": "octopus-ventures", "entity_def_id": "organization"}, "ipo_status": "private", "description": "For the people and ideas that will change the world, because we believe you can build a better tomorrow by investing in it.\n\nWe can back you at every stage of your journey, from ideas on a page through to IPO. Each year we deploy more than \u00a3200 million across five areas where we see maximum potential for impact: health, fintech, deep tech, consumer and B2B software. With many years of experience and expertise investing in each of these areas, our team can help our portfolio companies scale at speed.\n\nFor us, investment is about more than money. Being truly invested is about giving time, energy and focus; working with founders to give them the best chance of success. It means a shared commitment to solving the big problems that matter.\n\nWe\u2019ve backed the likes of ManyPets, Cazoo, Depop, Elvie and WaveOptics, while others have gone on to join forces with the world\u2019s largest businesses.\n\nWe are based in London and New York with a network of partners across the U.S and Asia.", "investor_stage": ["early_stage_venture", "late_stage_venture", "seed"], "category_groups": [{"uuid": "d7b67d9f-9b01-3864-1400-64fe3d4bebc3", "value": "Financial Services", "permalink": "financial-services-d7b6", "entity_def_id": "category_group"}, {"uuid": "e03218e3-0279-3faa-0151-558385ce5b94", "value": "Lending and Investments", "permalink": "lending-and-investments", "entity_def_id": "category_group"}], "linkedin": {"value": "https://www.linkedin.com/company/octopus-ventures?trk=biz-companies-cym"}, "short_description": "Octopus is a multi stage European venture capital investor specializing in health, fintech, deep tech, B2B software and consumer.", "num_portfolio_organizations": 245, "num_investments": 378, "revenue_range": "r_00001000", "operating_status": "active", "num_exits_ipo": 49, "num_employees_enum": "c_00051_00100", "status": "operating", "hub_tags": ["cbvp"], "categories": [{"uuid": "ae5718dc-8602-c733-e62b-28e6167091ad", "value": "Finance", "permalink": "finance", "entity_def_id": "category"}, {"uuid": "90b4194f-1d4f-ff5c-d7a6-6b6f32ae4892", "value": "Financial Services", "permalink": "financial-services", "entity_def_id": "category"}, {"uuid": "b37dbaeb-c825-e6a0-36de-84556a5d2e1f", "value": "Venture Capital", "permalink": "venture-capital-b37d", "entity_def_id": "category"}], "investor_type": ["venture_capital"], "created_at": "2014-06-06T16:57:12Z", "location_identifiers": [{"uuid": "aad17950-576b-8c44-8fd4-f44dbeb59220", "value": "London", "permalink": "london-england", "entity_def_id": "location", "location_type": "city"}, {"uuid": "79eb923b-9e93-e0db-2fe0-75f0c430c2cb", "value": "England", "permalink": "england-united-kingdom", "entity_def_id": "location", "location_type": "region"}, {"uuid": "a30e342c-1742-6b1c-66e9-461de680e54b", "value": "United Kingdom", "permalink": "united-kingdom", "entity_def_id": "location", "location_type": "country"}, {"uuid": "6106f5dc-823e-5da8-40d7-51612c0b2c4e", "value": "Europe", "permalink": "europe", "entity_def_id": "location", "location_type": "continent"}], "num_articles": 463, "num_event_appearances": 5, "twitter": {"value": "http://twitter.com/OctopusVentures"}, "rank_delta_d90": -2.8, "website_url": "http://www.octopusventures.com", "updated_at": "2022-07-19T03:02:21Z", "contact_email": "info@octopusvnetures.com", "num_exits": 49, "rank_delta_d7": -0.2}, {"first_name": "Patrick", "rank_delta_d30": -0.2, "image_url": "https://res.cloudinary.com/crunchbase-production/image/upload/edngsa68mbsaf5or6j9b", "rank_principal": 234074, "identifier": {"uuid": "8d851678-9164-e490-a3ae-7abcdb3f2287", "value": "Patrick Pichette", "image_id": "edngsa68mbsaf5or6j9b", "permalink": "patrick-pichette", "entity_def_id": "person"}, "primary_job_title": "General Partner", "description": "Patrick Pichette currently Board Member at LightSpeed and Twitter. He is a former Google's chief financial officer, where he started in June of 2008. He has nearly 20 years of experience in financial operations and management in the telecommunications sector, including seven years at Bell Canada, which he joined in 2001 as executive vice president of planning and performance management.\n\nDuring his time at Bell Canada, he held various executive positions, including CFO from 2002 until the end of 2003, and was instrumental in the management of the most extensive communications network in Canada and its ongoing migration to a new national IP-based infrastructure.\n\nPrior to joining Bell Canada, Patrick was a partner at McKinsey & Company, where he was a lead member of McKinsey's North American Telecom Practice. He also served as vice president and chief financial officer of Call-Net Enterprises, a Canadian telecommunications company.\n\nPatrick earned a bachelor's degree in business administration from UniversitA\u00a9 du Quebec A\u00a0 MontrA\u00a9al. He holds a master's degree in philosophy, politics, and economics from Oxford University, where he attended as a Rhodes Scholar. He also serves on the board of Engineers Without Borders (Canada).", "investor_stage": ["seed"], "linkedin": {"value": "https://www.linkedin.com/in/patrick-pichette-665836105"}, "short_description": "Patrick Pichette currently Board Member at LightSpeed and Twitter. He is a former Google's chief financial officer, where he started in June...", "num_partner_investments": 6, "num_portfolio_organizations": 10, "num_investments": 12, "num_jobs": 13, "num_exits_ipo": 1, "facebook": {"value": "https://www.facebook.com/Patrick.Pichette.D"}, "last_name": "Pichette", "investor_type": ["investment_partner", "angel"], "primary_organization": {"uuid": "488ba4ee-7e9b-9c4f-6550-493232dfcf58", "value": "Inovia Capital", "image_id": "z3elomvrx2ucg8iubvcm", "permalink": "inovia-capital", "entity_def_id": "organization"}, "created_at": "2008-06-27T08:30:25Z", "location_identifiers": [{"uuid": "aad17950-576b-8c44-8fd4-f44dbeb59220", "value": "London", "permalink": "london-england", "entity_def_id": "location", "location_type": "city"}, {"uuid": "79eb923b-9e93-e0db-2fe0-75f0c430c2cb", "value": "England", "permalink": "england-united-kingdom", "entity_def_id": "location", "location_type": "region"}, {"uuid": "a30e342c-1742-6b1c-66e9-461de680e54b", "value": "United Kingdom", "permalink": "united-kingdom", "entity_def_id": "location", "location_type": "country"}, {"uuid": "6106f5dc-823e-5da8-40d7-51612c0b2c4e", "value": "Europe", "permalink": "europe", "entity_def_id": "location", "location_type": "continent"}], "num_articles": 449, "num_event_appearances": 1, "twitter": {"value": "https://twitter.com/pichette"}, "rank_delta_d90": -0.2, "updated_at": "2022-02-25T11:58:22Z", "num_exits": 1, "rank_delta_d7": 0.2, "gender": "male"}, {"first_name": "Paul", "rank_delta_d30": -0.1, "image_url": "https://res.cloudinary.com/crunchbase-production/image/upload/zefgqo0adxsaeq044bkw", "rank_principal": 1485633, "identifier": {"uuid": "95a1f09e-6b4f-4aa9-8066-2215c949aaab", "value": "Paul Massara", "image_id": "zefgqo0adxsaeq044bkw", "permalink": "paul-massara-aaab", "entity_def_id": "person"}, "description": "Paul has extensive experience in the energy and service sectors in Europe and North America. Having stepped down as CEO of Electron he is now an Advisor to the Board, as well as being an Adviser to Habitat Energy, a member of the Committee on Fuel Poverty advising the UK Secretary of State for Energy, and Chair of Medicinema, a charity that builds cinemas in hospitals.\n\nHe has served on the Executive Committee of two of the largest utilities in Europe, Centrica Plc and RWE/INNOGY. From 2006 to 2009, he was President and Founding Partner of Genesis Capital Corporation, a Canadian private equity company investing funds in small and medium-sized industrial businesses. From 1997 to 2005, Paul held a variety of positions with increasing responsibility with Centrica plc, Direct Energy North America, in Toronto. His tenure there culminated in his role as President of Canadian Operations and serving as a Member of the North American Board and the North America Executive Committee. His leadership enhanced the Company\u2019s growth from $700,000MM to $2.7Bn, and from 600 to 3,000 employees.", "investor_stage": ["early_stage_venture", "seed"], "linkedin": {"value": "https://www.linkedin.com/in/paulmassara/"}, "short_description": "Paul has extensive experience in the energy and service sectors in Europe and North America. Having stepped down as CEO of Electron he is no...", "num_portfolio_organizations": 4, "num_investments": 4, "num_exits_ipo": 1, "last_name": "Massara", "investor_type": ["angel"], "created_at": "2020-05-13T20:49:37Z", "location_identifiers": [{"uuid": "c1af6713-07f3-7fef-0b8c-8ae1d7e71bb3", "value": "Oxford", "permalink": "oxford-oxfordshire", "entity_def_id": "location", "location_type": "city"}, {"uuid": "517ebbb4-6a4d-3a8a-41de-b8af68d48c69", "value": "Oxfordshire", "permalink": "oxfordshire-united-kingdom", "entity_def_id": "location", "location_type": "region"}, {"uuid": "a30e342c-1742-6b1c-66e9-461de680e54b", "value": "United Kingdom", "permalink": "united-kingdom", "entity_def_id": "location", "location_type": "country"}, {"uuid": "6106f5dc-823e-5da8-40d7-51612c0b2c4e", "value": "Europe", "permalink": "europe", "entity_def_id": "location", "location_type": "continent"}], "num_articles": 1, "rank_delta_d90": 4.5, "updated_at": "2020-05-13T20:49:37Z", "num_exits": 1, "rank_delta_d7": -0.1, "gender": "male"}, {"first_name": "Robert", "rank_delta_d30": -0.1, "image_url": "https://res.cloudinary.com/crunchbase-production/image/upload/v1469005050/a7xv7rcazqrtpnirtnlh.png", "rank_principal": 362914, "num_founded_organizations": 1, "identifier": {"uuid": "69294b81-b677-7b37-975a-9640bc1d938a", "value": "Robert Sansom", "image_id": "v1469005050/a7xv7rcazqrtpnirtnlh.png", "permalink": "robert-sansom", "entity_def_id": "person"}, "primary_job_title": "Founder", "description": "Dr. Robert Sansom is the Board Chairman of Netronome Systems Inc.\r\n\r\nHe is an active angel investor and mentor to start-up businesses in the UK and US. He is a Member of the Board of Directors of Azuro, Inc., a start-up developing tools to reduce power requirements on integrated circuits; Tenison Technology, which delivers software tools that enable hardware and software engineers designing integrated circuits to work together; Fetch, Inc., which delivers tools and services that integrate Internet-available data sources; and Ubisense, Ltd, which is the market leader in indoor, 3D location-detection systems. \r\n\r\nIn September 2001, he Co - Founded, and currently is Chairman of, Cambridge Angels, a group of seasoned technology and bio-technology entrepreneurs who invest in and mentor technology start-ups in the Cambridge area.\r\n\r\nIn 1990, Dr. Sansom Co - Founded FORE Systems, a data communications company which became the leading developer of ATM (Asynchronous Transfer Mode) networks. Dr. Sansom was initially responsible for FORE's finance and business strategy, including raising venture capital, then ran the software engineering group, building it up to a team of 200 engineers, and finally became responsible for technical strategy as Chief Technology Officer. \r\n\r\nDr. Sansom was also a Member of the Board of Directors of FORE. In 1999, when it employed over 2,000 employees and had annual revenues of over $600 million, FORE was sold to Marconi (previously GEC plc) for $4.5 billion. Dr. Sansom continued as Marconi\u2019s Chief Technology Officer until April 2000.\r\n\r\nDr. Sansom has a Ph.D. in Computer Science from Carnegie-Mellon University in Pittsburgh, USA.\r\n\r\nAn M.A. in Computer Science from the University of Cambridge, England.", "investor_stage": ["seed"], "linkedin": {"value": "https://www.linkedin.com/in/rsansom/"}, "short_description": "Dr. Robert Sansom is the Board Chairman of Netronome Systems Inc.\r\n\r\nHe is an active angel investor and mentor to start-up businesses in the...", "num_partner_investments": 6, "num_portfolio_organizations": 5, "num_investments": 5, "num_jobs": 16, "facebook": {"value": "https://www.facebook.com/RobertDSansom"}, "last_name": "Sansom", "investor_type": ["investment_partner", "angel"], "primary_organization": {"uuid": "af74e34b-fb16-94d5-a879-ad3698edede6", "value": "Cambridge Angels group", "image_id": "v1397178978/bb18ff84c7bcbefc80b6f7ac436fd445.jpg", "permalink": "cambridge-angels-group", "entity_def_id": "organization"}, "created_at": "2009-12-20T07:30:28Z", "location_identifiers": [{"uuid": "f2e1d95c-c47d-8e7f-90eb-5c2ff71599e2", "value": "Cambridge", "permalink": "cambridge-cambridgeshire", "entity_def_id": "location", "location_type": "city"}, {"uuid": "5038b7a6-2cef-2259-1b8e-66df18858d06", "value": "Cambridgeshire", "permalink": "cambridgeshire-united-kingdom", "entity_def_id": "location", "location_type": "region"}], "num_articles": 5, "twitter": {"value": "https://twitter.com/rsansom"}, "rank_delta_d90": 2, "updated_at": "2021-06-18T06:09:18Z", "rank_delta_d7": -0.1, "gender": "male"}, {"location_group_identifiers": [{"uuid": "c4fcb2dd-eee8-4140-bd21-afe31b1e28a1", "value": "Nordic Countries", "permalink": "nordic-countries", "entity_def_id": "location", "location_type": "group"}, {"uuid": "5aae468d-1308-44a8-9aa7-f92715ef12f6", "value": "Scandinavia", "permalink": "scandinavia", "entity_def_id": "location", "location_type": "group"}], "rank_delta_d30": 1.8, "image_url": "https://res.cloudinary.com/crunchbase-production/image/upload/v1431928192/dn2s3anncilsctiyqsxk.png", "founded_on": {"value": "1986-01-01", "precision": "day"}, "website": {"value": "http://tdveen.no/en/"}, "num_lead_investments": 11, "rank_principal": 232427, "identifier": {"uuid": "0cf953b7-00bb-e607-e775-f781b1b7cf51", "value": "TD Veen", "image_id": "v1431928192/dn2s3anncilsctiyqsxk.png", "permalink": "td-veen", "entity_def_id": "organization"}, "ipo_status": "private", "founder_identifiers": [{"uuid": "3c716fc4-7ef2-48c8-b28d-c2b18e9b366d", "value": "SILJE VEEN", "image_id": "qwlpknaedaigwreujwhe", "permalink": "silje-veen", "entity_def_id": "person"}, {"uuid": "9cd628cb-9e6b-489e-a3ba-7da28ef089db", "value": "TOR DAGFINN VEEN", "image_id": "uqduevo9c7s92uspqivp", "permalink": "tor-dagfinn-veen", "entity_def_id": "person"}], "description": "TD Veen is concerned with sustainable social development. Their contribution is to invest in companies and solutions that are good for both owners and society at large. The foundation of TD Veen is job satisfaction, creativity and a strong determination to invest in the very best people.\r\n\r\nAt one stage they saw sense in targeting the investment profile, which also gave us a new perspective on the demand for returns and the role of investors. As a financial owner, they want to establish a healthy attitude toward results and returns. In companies where value is created over time.\r\n\r\nA differentiated portfolio is important to us. That is why they work with investments in new industries, such as e.g. health, or in parts of the world where they have not been before. Despite their Stavanger address, they want to invest outside of the petroleum sector.\r\nTD Veen has for many years supported the work of SOS Children\u2019s Villages, including the project The Long Run, where they renovated old children\u2019s villages in Romania and Lebanon. They have also contributed to starting up a separate pioneer project for foster families in Murmansk. Going ahead, they will to a greater degree also contribute to society at home, by building up a community of social entrepreneurs in Eiganesveien 95.", "short_description": "TD Veen is concerned with sustainable social development.", "num_portfolio_organizations": 18, "num_investments": 19, "revenue_range": "r_00001000", "operating_status": "active", "num_exits_ipo": 5, "num_employees_enum": "c_00001_00010", "status": "operating", "investor_type": ["venture_capital"], "created_at": "2015-05-18T05:50:56Z", "location_identifiers": [{"uuid": "448d887d-1bee-0a6b-51bf-9afadc05a838", "value": "Stavanger", "permalink": "stavanger-rogaland", "entity_def_id": "location", "location_type": "city"}, {"uuid": "c95ccb32-22dd-bfd1-41d0-4f11a5208059", "value": "Rogaland", "permalink": "rogaland-norway", "entity_def_id": "location", "location_type": "region"}, {"uuid": "9ae73d37-96ed-28a8-46ea-f52b26a2f97d", "value": "Norway", "permalink": "norway", "entity_def_id": "location", "location_type": "country"}, {"uuid": "6106f5dc-823e-5da8-40d7-51612c0b2c4e", "value": "Europe", "permalink": "europe", "entity_def_id": "location", "location_type": "continent"}], "num_articles": 1, "rank_delta_d90": -0.5, "website_url": "http://tdveen.no/en/", "updated_at": "2023-06-22T05:34:23Z", "num_exits": 5, "rank_delta_d7": 0, "num_founders": 2}, {"first_name": "William", "rank_delta_d30": -0.5, "image_url": "https://res.cloudinary.com/crunchbase-production/image/upload/kuhdy2jn93u8i5zmngd5", "website": {"value": "https://williamtp.com/"}, "rank_principal": 241324, "num_diversity_spotlight_investments": 1, "num_founded_organizations": 3, "identifier": {"uuid": "2d90af63-9abb-fa96-03cd-a5c7e897579c", "value": "William Tunstall-Pedoe", "image_id": "kuhdy2jn93u8i5zmngd5", "permalink": "william-tunstall-pedoe", "entity_def_id": "person"}, "description": "William Tunstall-Pedoe FREng is a British entrepreneur focused on world-changing products powered with Artificial Intelligence and other deep technology. He is based in Cambridge and London UK. He founded the British company Evi (formerly True Knowledge), the AI platform acquired by Amazon and where William, the team and the technology developed by Evi was used to create Alexa. He left in 2016. William is an active angel investor with investments in more than 100 startups. He is a fellow of the Creative Destruction Lab and a full member of Cambridge Angels.", "investor_stage": ["seed"], "linkedin": {"value": "http://www.linkedin.com/in/williamtp"}, "short_description": "William Tunstall-Pedoe FREng is a British entrepreneur focused on world-changing products powered with Artificial Intelligence and other dee...", "num_portfolio_organizations": 14, "num_investments": 15, "num_jobs": 7, "num_exits_ipo": 1, "last_name": "Tunstall-Pedoe", "investor_type": ["angel"], "created_at": "2007-11-08T18:22:38Z", "location_identifiers": [{"uuid": "f2e1d95c-c47d-8e7f-90eb-5c2ff71599e2", "value": "Cambridge", "permalink": "cambridge-cambridgeshire", "entity_def_id": "location", "location_type": "city"}, {"uuid": "5038b7a6-2cef-2259-1b8e-66df18858d06", "value": "Cambridgeshire", "permalink": "cambridgeshire-united-kingdom", "entity_def_id": "location", "location_type": "region"}], "num_articles": 46, "num_event_appearances": 1, "twitter": {"value": "http://twitter.com/williamtp"}, "rank_delta_d90": -0.4, "website_url": "https://williamtp.com/", "updated_at": "2022-09-19T12:59:32Z", "num_exits": 1, "rank_delta_d7": -0.1, "gender": "male"}], "acquiree_acquisitions": [], "participated_funding_rounds": []}, "source": "crunchbase"}"""
crunchbase_sample = ujson.loads(crunchbase_sample)
linkedin_sample = """{"parsing_date": "2023-08-22", "uuid": "c4a15ea3-b525-5da9-a6a8-03beb1fe3fc2", "parent_uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "companieshouse_id": "10388803", "name": "Djordje Djokovic", "full_name": "Djordje Djokovic", "linkedin_name": "Djordje Djokovic, PhD", "companieshouse_company_name": "Isize Limited", "occupations": ["Founder", "Shareholder"], "linkedin_url": "https://www.linkedin.com/in/djordje-djokovic-phd-683589", "country": "GB", "city": "", "experience": [{"items": [{"position": "Trader", "from": "2013-12-01", "to": "2021-10-01", "days": 2861, "country": "GB", "city": "London", "description": "Heading Automated Options Market Making"}], "organization": "Maven Securities", "from": "2013-12-01", "to": "2021-10-01", "days": 2861}, {"items": [{"position": "Co-Founder and Shareholder", "from": "2016-09-01", "to": "2021-03-01", "days": 1642, "country": "GB", "city": "London", "description": ""}], "organization": "iSIZE Technologies - Enhancing Video Streaming With Advanced Machine Learning", "from": "2016-09-01", "to": "2021-03-01", "days": 1642}, {"items": [{"position": "Quantitative Trader", "from": "2007-03-01", "to": "2013-07-01", "days": 2314, "country": "GB", "city": "London", "description": "Heading the implementation and trading of a profitable automated option market making business. Engineered and implemented a fully automated options market making system including:- Fully automated and stable volatility fitting solution- Intelligent option quote and hitting strategies- Advanced futures hedging strategies- Real time correlation between multiple products/ volatility surfaces- Currently simultaneously trading DAX and ESX index options on EUREX)Responsible for implementing high frequency infrastructure such as fast and efficient analytic libraries, economic tick data storage solutions and matching engine simulators for backtesting purposes.Managed global quantitative model implementations across global exchanges and our London, Chicago, Sydney and Hong Kong offices."}], "organization": "Liquid Capital Group", "from": "2007-03-01", "to": "2013-07-01", "days": 2314}, {"items": [{"position": "Algorithmic Trading Associate", "from": "2005-03-01", "to": "2007-12-01", "days": 1005, "country": "GB", "city": "London", "description": "Co-Developed automatic high frequency multi-asset trading platform including infrastructure and key pricing and order algorithms. The system was managing a daily trade frequency of around 5000 trades per day in more than 1000 securities (equity and fixed income).Trading and managing own high frequency equity book, monitoring trading platform performance and generating and implementing new algorithm ideas.Implemented backtesting engine,  strategy optimizer and responsible for development and testing of proprietary elements"}], "organization": "Dresdner Kleinwort", "from": "2005-03-01", "to": "2007-12-01", "days": 1005}], "education": [{"items": [{"degree": "PhD, Pricing of Real Options", "from": "2003-01-01", "to": "2010-01-01", "days": 2557, "country": "", "city": "", "description": "Pricing of patents and high technologies in uncertain markets using real option theory"}], "organization": "Cass Business School", "from": "2003-01-01", "to": "2010-01-01", "days": 2557}, {"items": [{"degree": "MSc Material Science, Nanotechnology", "from": "2002-01-01", "to": "2003-01-01", "days": 365, "country": "", "city": "", "description": ""}], "organization": "University of Hawaii at Manoa", "from": "2002-01-01", "to": "2003-01-01", "days": 365}, {"items": [{"degree": "Diplom Ingenieur, Mechanical Engineering", "from": "1996-01-01", "to": "2001-01-01", "days": 1827, "country": "", "city": "", "description": ""}], "organization": "RWTH Aachen University", "from": "1996-01-01", "to": "2001-01-01", "days": 1827}]}"""
linkedin_sample = ujson.loads(linkedin_sample)
# json.dumps(crunchbase_sample)
json.dumps(crunchbase_sample['cards']['raised_investments'])

'[{"is_lead_investor": true, "identifier": {"uuid": "08d98362-458a-4340-813e-edc8ba09c212", "value": "Octopus Ventures investment in Series A - iSIZE Technologies", "permalink": "octopus-ventures-invested-in-isize-technologies-series-a--b5bba780--08d98362", "entity_def_id": "investment"}, "partner_identifiers": [{"uuid": "a982c120-c0d8-6745-30ff-8e42f3bace05", "value": "Simon King", "image_id": "v1498115333/piewmmrycy625vn57x4y.png", "permalink": "simon-king-2", "entity_def_id": "person"}], "investor_stage": ["early_stage_venture", "late_stage_venture", "seed"], "announced_on": "2021-05-06", "organization_identifier": {"uuid": "a7d2f427-66ba-476a-81cc-171a5d806b22", "value": "iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies", "entity_def_id": "organization"}, "funding_round_identifier": {"uuid": "b5bba780-737a-492a-98e3-9298e2a4266e", "value": "Series A - iSIZE Technologies", "image_id": "xmuwkvi2citoghycs1tu", "permalink": "isize-technologies-s

In [4]:
unique_shareholders = get_unique_shareholders(companieshouse_sample['cards']['shareholding'])

In [5]:
unique_officers = get_unique_officers(companieshouse_sample['cards']['officer']['items'])

In [6]:
unique_founders = get_unique_founders(companieshouse_sample['cards']['incorporation']['items'])

In [7]:
unique_entities = get_persons(data=companieshouse_sample, ignore_organization=False)

In [9]:
def get_companieshouse_data_with_unique_names(unique_entities, companieshouse_data):
    data = copy.deepcopy(companieshouse_data)
    # Retrieve the parsing date from the data's properties
    parsing_date = data['properties']['parsing_date']
    # Initialize the last shareholder update to the parsing date
    last_shareholder_date = parsing_date
    
    # Make deep copies of the unique entities and different sections of the data
    # This is done to prevent modifying the original data structures during updates
    unique_entities_copy = copy.deepcopy(unique_entities['items'])
    
    # Copy the 'shareholding' section for updating
    shareholding_update = copy.deepcopy(data['cards']['shareholding'])
    # Copy the 'incorporation' section for updating
    incorporation_update = copy.deepcopy(data['cards']['incorporation']['items'])
    # Copy the 'officer' section for updating
    officer_update = copy.deepcopy(data['cards']['officer']['items'])
    
    # Create copies of the original data for checking updates later
    shareholding_check = copy.deepcopy(data['cards']['shareholding'])
    incorporation_check = copy.deepcopy(data['cards']['incorporation']['items'])
    officer_check = copy.deepcopy(data['cards']['officer']['items'])
    
    # Initialize an index to track the position within unique entities
    index = 0
    # Iterate over each unique entity in the list of entities
    for entity in unique_entities['items']:
        # Iterate over each occupation listed for the entity
        for occupation in entity['occupation']:
            
            # Check if the occupation is 'Shareholder'
            if occupation == 'Shareholder':
    
                # Iterate over the 'shareholding' data items
                for received_date, item in data['cards']['shareholding'].items():
                    
                    # Retrieve the filing date from the current item
                    filing_date = start_date = item['filing_date']
                    # Initialize a total shares counter
                    total_shares = 0
                    # Sum up the total shares from all share items
                    for x in item['items']:
                        total_shares += x['shares']
                    
                    # Initialize a list to keep track of shareholders that will be deleted
                    shareholders_to_delete = []
    
                    # Initialize an index for shareholders
                    i_shareholder = 0
                    # Iterate over each shareholder in the current item
                    for shareholder in item['items']:
                        # Check if the shareholder's name is an alias of the entity
                        if shareholder['name'] in entity['alias']:
                            
                            # Update the shareholder's information with the entity's details
                            shareholder_update = shareholding_update[received_date]['items'][i_shareholder]
                            shareholder_update['name_original'] = shareholder['name']
                            shareholder_update['name'] = entity['name']
                            shareholder_update['uuid'] = entity['uuid']
                            shareholder_update['is_organization'] = entity['is_organization']
                            del shareholder_update['is_company']
                            
                            # Add this shareholder to the list of shareholders to delete
                            shareholders_to_delete.append(shareholder) 
                        
                        # Increment the shareholder index
                        i_shareholder += 1
                    
                    # Check if the received date is in the shareholding check list
                    if received_date in shareholding_check:    
                        # Iterate over the shareholders to delete
                        for shareholder in shareholders_to_delete:
                            # Remove the shareholder from the check list
                            shareholding_check[received_date]['items'].remove(shareholder)
    
                        # If there are no items left after deletion, remove the item completely
                        if not shareholding_check[received_date]['items']:
                            del shareholding_check[received_date]
                        
                    # Update the last shareholder date with the filing date
                    last_shareholder_date = filing_date
            # Placeholders for other occupation cases (Director, Founder, Secretary)
            elif occupation == 'Director':
                i_director = 0
                for director in data['cards']['officer']['items']:            
                    if director['name'] in entity['alias']:
                        director_update = officer_update[i_director]
                        director_update['name_original'] = director['name']
                        director_update['name'] = entity['name']
                        director_update['uuid'] = entity['uuid']
                        director_update['is_organization'] = entity['is_organization']
                        # del director_update['is_company']
                        officer_check.remove(director)
                    i_director += 1
            elif occupation == 'Secretary':
                i_secretary = 0
                for secretary in data['cards']['officer']['items']:            
                    if secretary['name'] in entity['alias']:
                        secretary_update = officer_update[i_secretary]
                        secretary_update['name_original'] = secretary['name']
                        secretary_update['name'] = entity['name']
                        secretary_update['uuid'] = entity['uuid']
                        secretary_update['is_organization'] = entity['is_organization']
                        # del secretary_update['is_company']
                        officer_check.remove(secretary)
                    i_secretary += 1
            elif occupation == 'Founder':
                i_founder = 0
                for founder in data['cards']['incorporation']['items']:            
                    if founder['name'] in entity['alias']:
                        founder_update = incorporation_update[i_founder]
                        founder_update['name_original'] = founder['name']
                        founder_update['name'] = entity['name']
                        founder_update['uuid'] = entity['uuid']
                        founder_update['is_organization'] = entity['is_organization']
                        del founder_update['is_company']
                        incorporation_check.remove(founder)
                    
                    i_founder += 1
            else:
                # Raise an error if there is an undefined occupation
                raise NotImplementedError()
            # Remove the processed occupation from the entity's copy
            unique_entities_copy[index]['occupation'].remove(occupation)
        # Increment the index for the next entity
        index += 1
    
    # Assertions to ensure that all occupations are processed and shareholdings are updated
    assert(len([d for d in unique_entities_copy if d['occupation'] != []])==0)
    assert(len(shareholding_check)==0)
    assert(len(incorporation_check)==0)
    assert(len(officer_check)==0)
    
    # Sort the updated shareholding items by name for each date received
    for received_date, item in shareholding_update.items():
        item["items"].sort(key=lambda x: x["name"])
    
    # Sort the updated founders items by name
    incorporation_update.sort(key=lambda x: x["name"])
    
    # Sort the updated founders items by name
    officer_update.sort(key=lambda x: x["name"])
    
    data['cards']['shareholding'] = shareholding_update
    data['cards']['incorporation']['items'] = incorporation_update
    data['cards']['officer']['items'] = officer_update
    
    return data

In [10]:
companieshouse_sample_unique = get_companieshouse_data_with_unique_names(unique_entities, companieshouse_sample)

In [ ]:
companieshouse_sample_unique

In [42]:
def flatten_companieshouse_data(companieshouse_data_unique):
    def create_dict(uuid, name, parent_name, parent_uuid, group_id, start_date, end_date, variable, value):
        return {
            'uuid': uuid, 
                    'name': name, 
            'source': 'companieshouse', 
                    'parent_uuid': parent_uuid, 
                    'parent_name': parent_name, 
                    'group_id': group_id, 
                    'start_date': start_date, 
                    'end_date': end_date, 
                    'variable': variable, 
                    'value': value}
    
    data = companieshouse_data_unique
    last_filing_date = None
    unique_uuids = set()
    entries = []
    
    parent_name = data['properties']['company_name']
    parent_uuid = data['properties']['uuid']

    
    insolvency_date = None
    if data['cards']['insolvency']['items']:
        raise NotImplementedError('we also need to record dissolved date which is not part of insolvency')
    elif 'dissolved_on' in data['properties'] and data['properties']['dissolved_on'] is not None:
        entries.append(create_dict(parent_uuid, parent_name, None, None, None, data['properties']['incorporated_on'], data['properties']['dissolved_on'], 'is_incorporated', True))
        entries.append(create_dict(parent_uuid, parent_name, None, None, None, data['properties']['incorporated_on'], data['properties']['dissolved_on'], 'is_dissolved', True))
    else:
        entries.append(create_dict(parent_uuid, parent_name, None, None, None, data['properties']['incorporated_on'], None, 'is_incorporated', True))
       
    for received_date, item in data['cards']['shareholding'].items():
        
        # print(received_date, item)
        filing_date = start_date = item['filing_date']
        total_shares = 0
        
        for x in item['items']:
            total_shares += x['shares']
    
        # add total_shares
        entries.append(create_dict(parent_uuid, parent_name, None, None, None, filing_date, last_filing_date, 'total_shares', total_shares))
        unique_shareholders = set()
        i_unique_shareholding = 0
        for shareholder in item['items']:
            uuid = shareholder['uuid']
            name = shareholder['name']
            shares = shareholder['shares']
            share_type = shareholder['share_type']
            
            if uuid not in unique_uuids:
                unique_uuids.add(uuid)
                i_unique_shareholding += 1
                # add is_organization
                if shareholder['is_organization']:
                    entries.append(create_dict(uuid, name, parent_uuid, parent_name, None, data['cards']['incorporation']['received_date'], None, 'is_organization', True))
                else:
                    entries.append(create_dict(uuid, name, parent_uuid, parent_name, None, data['cards']['incorporation']['received_date'], None, 'is_organization', False))
    
            if i_unique_shareholding == 0:
                group_id = 'shareholding'
            else:
                group_id = uuid #f'shareholding_{i_unique_shareholding}'
                
            entries.append(create_dict(uuid, name, parent_uuid, parent_name, group_id, filing_date, last_filing_date, 'shareholding', shares/ total_shares))
            entries.append(create_dict(uuid, name, parent_uuid, parent_name, group_id, filing_date, last_filing_date, 'share_type', share_type))
            entries.append(create_dict(uuid, name, parent_uuid, parent_name, group_id, filing_date, last_filing_date, 'share_number', shares))
    
    for officer in data['cards']['officer']['items']:
        uuid = officer['uuid']
        name = officer['name']
        start_date = officer['appointed_on']
        end_date = officer['resigned_on']
    
        if start_date is None:
            raise NotImplementedError()
        if end_date is None:
            end_date = None
    
        if officer['role'] == 'Director':
            entries.append(create_dict(uuid, name, parent_uuid, parent_name, None, start_date, end_date, 'is_director', True))
        elif officer['role'] == 'Secretary':
            entries.append(create_dict(uuid, name, parent_uuid, parent_name, None, start_date, end_date, 'is_secretary', True))
        else:
            raise NotImplementedError()
    
    for founder in data['cards']['incorporation']['items']:
        uuid = founder['uuid']
        name = founder['name']
        start_date = data['cards']['incorporation']['received_date']
        end_date = None
    
        entries.append(create_dict(uuid, name, parent_uuid, parent_name, None, start_date, end_date, 'is_founder', True))
    return entries

In [43]:
company_uuid = companieshouse_sample_unique['properties']['uuid']
company_name = companieshouse_sample_unique['properties']['company_name']
entries = flatten_companieshouse_data(companieshouse_sample_unique)

In [57]:
pd.DataFrame(entries).head(20)

,uuid,name,source,parent_uuid,parent_name,group_id,start_date,end_date,variable,value
0,a7d2f427-66ba-476a-81cc-171a5d806b22,Isize Limited,companieshouse,None,None,None,2016-09-22,None,is_incorporated,True
1,a7d2f427-66ba-476a-81cc-171a5d806b22,Isize Limited,companieshouse,None,None,None,2022-09-21,None,total_shares,2192636
2,47b0622a-1e11-558a-a4f4-8c63205135ee,A Management As,companieshouse,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,None,2016-09-22,None,is_organization,True
3,47b0622a-1e11-558a-a4f4-8c63205135ee,A Management As,companieshouse,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,47b0622a-1e11-558a-a4f4-8c63205135ee,2022-09-21,None,shareholding,0.001778
4,47b0622a-1e11-558a-a4f4-8c63205135ee,A Management As,companieshouse,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,47b0622a-1e11-558a-a4f4-8c63205135ee,2022-09-21,None,share_type,ORDINARY
5,47b0622a-1e11-558a-a4f4-8c63205135ee,A Management As,companieshouse,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,47b0622a-1e11-558a-a4f4-8c63205135ee,2022-09-21,None,share_number,3898
6,47b0622a-1e11-558a-a4f4-8c63205135ee,A Management As,companieshouse,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,47b0622a-1e11-558a-a4f4-8c63205135ee,2022-09-21,None,shareholding,0.000933
7,47b0622a-1e11-558a-a4f4-8c63205135ee,A Management As,companieshouse,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,47b0622a-1e11-558a-a4f4-8c63205135ee,2022-09-21,None,share_type,SERIES SEED 1 ORDINARY
8,47b0622a-1e11-558a-a4f4-8c63205135ee,A Management As,companieshouse,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,47b0622a-1e11-558a-a4f4-8c63205135ee,2022-09-21,None,share_number,2046
9,16d25409-6ae1-5b3a-a0fe-1a59911dccb5,Adriana Belonic,companieshouse,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,None,2016-09-22,None,is_organization,False


In [70]:
import pandas as pd
def flatten_crunchbase_properties(crunchbase_sample, company_uuid, company_name):
    # Provided dataset
    data = crunchbase_sample['cards']['fields']
    # Function to flatten nested dictionaries for category_groups and categories
    def flatten_categories(data, key):
        return [{'group_id': cat['uuid'], 'value': cat['value']} for cat in data.get(key, [])]
    
    # Initialize a list to hold all the rows
    rows = []
    
    # Function to add rows to list
    def add_row(rows, id_value, name_value, variable, value, group_id=None):
        rows.append({
            'uuid': id_value,
            'name': name_value,
            'source': 'crunchbase',
            'parent_id': None,
            'parent_name': None,
            'group_id': group_id,
            'variable': variable,
            'from_date': data['founded_on']['value'],
            'to_date': None,
            'value': value
        })
    
    # Extracting data from the provided dictionary and adding rows to list
    id_value = company_uuid # data['identifier']['uuid']
    name_value = company_name #data['identifier']['value']
    
    variables_to_extract = {
        'contact_email': data.get('contact_email', None),
        'website_url': data.get('website_url', None),
        'twitter': data.get('twitter', {}).get('value', None),
        'linkedin': data.get('linkedin', {}).get('value', None),
        'founded_on': data.get('founded_on', {}).get('value', None),
        'equity_funding_total': data.get('equity_funding_total', {}).get('value_usd', None),
        'entity': data.get('identifier', {}).get('entity_def_id', None),
        'ipo_status': data.get('ipo_status', None),
        'description': data.get('description', None),
        'num_investors': data.get('num_investors', None),
        'num_lead_investors': data.get('num_lead_investors', None),
        'num_employees_enum': data.get('num_employees_enum', None),
        'status': data.get('status', None),
        'funding_stage': data.get('funding_stage', None),
        'num_articles': data.get('num_articles', None),
        'num_funding_rounds': data.get('num_funding_rounds', None),
        'last_equity_funding_type': data.get('last_equity_funding_type', None)
    }
    
    # Add variables to list
    for variable, value in variables_to_extract.items():
        add_row(rows, id_value, name_value, variable, value)
    
    # Add category_groups and categories to list
    for group in flatten_categories(data, 'category_groups'):
        add_row(rows, id_value, name_value, 'category_groups', group['value'], group['group_id'])
    
    for category in flatten_categories(data, 'categories'):
        add_row(rows, id_value, name_value, 'categories', category['value'], category['group_id'])
    
    # Convert list to DataFrame
    df = pd.DataFrame(rows)
    
    # Show the first few rows of the dataframe
    return df

In [71]:
flatten_crunchbase_properties(crunchbase_sample, company_name, company_uuid)

,uuid,name,source,parent_id,parent_name,group_id,variable,from_date,to_date,value
0,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,crunchbase,None,None,None,contact_email,2016-09-22,None,info@isize.co
1,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,crunchbase,None,None,None,website_url,2016-09-22,None,https://www.isize.co
2,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,crunchbase,None,None,None,twitter,2016-09-22,None,https://twitter.com/isize_co
3,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,crunchbase,None,None,None,linkedin,2016-09-22,None,https://www.linkedin.com/company/isize.co/
4,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,crunchbase,None,None,None,founded_on,2016-09-22,None,2016-09-22
5,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,crunchbase,None,None,None,equity_funding_total,2016-09-22,None,7942415
6,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,crunchbase,None,None,None,entity,2016-09-22,None,organization
7,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,crunchbase,None,None,None,ipo_status,2016-09-22,None,private
8,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,crunchbase,None,None,None,description,2016-09-22,None,iSIZE Technologies is a company to delivers de...
9,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,crunchbase,None,None,None,num_investors,2016-09-22,None,9


In [68]:
def flatten_crunchbase_investments(crunchbase_sample, company_name, company_uuid):
    investments = crunchbase_sample['cards']['raised_investments']
    # Initialize an empty list to collect data
    data_rows = []
    
    # Iterate through the investment data to populate the list
    for investment in investments:
        # Common data for all rows related to this investment
        common_data = {
            'uuid': investment['funding_round_identifier']['uuid'],
            'name': investment['funding_round_identifier']['value'].split('-')[0].strip(),
            'parent_uuid': company_uuid, #investment['organization_identifier']['uuid'],
            'parent_name': company_name,        
            'group_id': None, #investment['investor_identifier']['uuid'],  # This could be the investor's UUID
            'start_date': investment.get('announced_on'),
            'end_date': None,
            'variable': '',
            'value': None
        }
    
        # Number of investors for the round
        common_data['variable'] = 'num_investors'
        common_data['value'] = 1  # Each entry is an investment, so we count 1 per entry
        data_rows.append(common_data.copy())
    
        # Other details
        details = [
            ('total_investment_usd', investment['funding_round_money_raised'].get('value_usd')),
            ('investor_type', ', '.join(investment.get('investor_type', []))),
            ('investor_stage', ', '.join(investment.get('investor_stage', []))),
            ('lead_investor', investment.get('is_lead_investor')),
            ('funding_round_investment_type', investment.get('funding_round_investment_type'))
        ]
        
        for variable, value in details:
            common_data['variable'] = variable
            common_data['value'] = value
            data_rows.append(common_data.copy())
    
    # Now, we need to aggregate the number of investors and total investment by round
    # To do this, we'll create a DataFrame and then perform groupby operations
    df = pd.DataFrame(data_rows)
    # Aggregate the number of investors by round
    df_num_investors = df[df['variable'] == 'num_investors'].groupby(['uuid', 'name', 'parent_uuid', 'parent_name', 'group_id'])['value'].sum().reset_index()
    df_num_investors['variable'] = 'num_investors'
    
    # For total investment, we just need to sum up the values by round
    df_total_investment = df[df['variable'] == 'total_investment_usd'].groupby(['uuid', 'name', 'parent_uuid', 'parent_name', 'group_id'])['value'].sum().reset_index()
    df_total_investment['variable'] = 'total_investment_usd'
    
    # Now we can concatenate these aggregated results back to the main DataFrame
    df_aggregated = pd.concat([df, df_num_investors, df_total_investment], ignore_index=True)
    
    # Finally, we filter out the rows that were used for aggregation (they are now redundant)
    df_aggregated = df_aggregated.drop_duplicates(subset=['uuid', 'variable'], keep='last')
    
    # Sort the DataFrame for better readability
    df_aggregated.sort_values(by=['uuid', 'variable'], inplace=True)
    
    # Display the first few rows of the DataFrame to verify
    df_aggregated
    return df_aggregated

In [69]:
flatten_crunchbase_investments(crunchbase_sample, company_uuid, company_name)

,uuid,name,parent_uuid,parent_name,group_id,start_date,end_date,variable,value
47,423015a6-011f-4e54-8cbc-c9fe0c2e2298,Seed Round,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,None,2020-05-11,None,funding_round_investment_type,seed
45,423015a6-011f-4e54-8cbc-c9fe0c2e2298,Seed Round,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,None,2020-05-11,None,investor_stage,seed
44,423015a6-011f-4e54-8cbc-c9fe0c2e2298,Seed Round,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,None,2020-05-11,None,investor_type,angel
46,423015a6-011f-4e54-8cbc-c9fe0c2e2298,Seed Round,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,None,2020-05-11,None,lead_investor,None
42,423015a6-011f-4e54-8cbc-c9fe0c2e2298,Seed Round,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,None,2020-05-11,None,num_investors,1
43,423015a6-011f-4e54-8cbc-c9fe0c2e2298,Seed Round,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,None,2020-05-11,None,total_investment_usd,912549
65,9766f01c-6b7c-4cd9-bf19-8a29e64b8806,Pre Seed Round,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,None,2018-03-01,None,funding_round_investment_type,pre_seed
63,9766f01c-6b7c-4cd9-bf19-8a29e64b8806,Pre Seed Round,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,None,2018-03-01,None,investor_stage,seed
62,9766f01c-6b7c-4cd9-bf19-8a29e64b8806,Pre Seed Round,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,None,2018-03-01,None,investor_type,"investment_partner, angel"
64,9766f01c-6b7c-4cd9-bf19-8a29e64b8806,Pre Seed Round,Isize Limited,a7d2f427-66ba-476a-81cc-171a5d806b22,None,2018-03-01,None,lead_investor,None


In [84]:
import pandas as pd

def flatten_linkedin(profile_data):
    # Initialize a list to store each flattened record
    flattened_records = []

    # Flatten experience data
    for experience in profile_data['experience']:
        # Add a record for the organization of the experience
        flattened_records.append({
            'uuid': profile_data['uuid'],
            'name': profile_data['name'],
            'parent_uuid': profile_data['parent_uuid'],
            'parent_name': profile_data['companieshouse_company_name'],
            'group_id': None, 
            'start_date': experience['from'],
            'end_date': experience['to'],
            'variable': 'experience_organization',
            'value': experience['organization']
        })
        # Add records for each item in the experience
        for item in experience['items']:
            flattened_records.extend([
                {
                    'uuid': profile_data['uuid'],
                    'name': profile_data['name'],
                    'parent_uuid': profile_data['parent_uuid'],
                    'parent_name': profile_data['companieshouse_company_name'],
                    'group_id': None, 
                    'start_date': item['from'],
                    'end_date': item['to'],
                    'variable': 'experience_position',
                    'value': item['position']
                },
                {
                    'uuid': profile_data['uuid'],
                    'name': profile_data['name'],
                    'parent_uuid': profile_data['parent_uuid'],
                    'parent_name': profile_data['companieshouse_company_name'],
                    'group_id': None, 
                    'start_date': item['from'],
                    'end_date': item['to'],
                    'variable': 'experience_country',
                    'value': item['country']
                },
                {
                    'uuid': profile_data['uuid'],
                    'name': profile_data['name'],
                    'parent_uuid': profile_data['parent_uuid'],
                    'parent_name': profile_data['companieshouse_company_name'],
                    'group_id': None, 
                    'start_date': item['from'],
                    'end_date': item['to'],
                    'variable': 'experience_city',
                    'value': item['city']
                },
                {
                    'uuid': profile_data['uuid'],
                    'name': profile_data['name'],
                    'parent_uuid': profile_data['parent_uuid'],
                    'parent_name': profile_data['companieshouse_company_name'],
                    'group_id': None, 
                    'start_date': item['from'],
                    'end_date': item['to'],
                    'variable': 'experience_position_description',
                    'value': item['description']
                }
            ])

    # Flatten education data
    for education in profile_data['education']:
        # Add a record for the organization of the education
        flattened_records.append({
            'uuid': profile_data['uuid'],
            'name': profile_data['name'],
            'parent_uuid': profile_data['parent_uuid'],
            'parent_name': profile_data['companieshouse_company_name'],
            'group_id': None, 
            'start_date': education['from'],
            'end_date': education['to'],
            'variable': 'education_organization',
            'value': education['organization']
        })
        # Add records for each item in the education
        for item in education['items']:
            flattened_records.extend([
                {
                    'uuid': profile_data['uuid'],
                    'name': profile_data['name'],
                    'parent_uuid': profile_data['parent_uuid'],
                    'parent_name': profile_data['companieshouse_company_name'],
                    'group_id': None, 
                    'start_date': item['from'],
                    'end_date': item['to'],
                    'variable': 'education_degree',
                    'value': item['degree']
                },
                {
                    'uuid': profile_data['uuid'],
                    'name': profile_data['name'],
                    'parent_uuid': profile_data['parent_uuid'],
                    'parent_name': profile_data['companieshouse_company_name'],
                    'group_id': None, 
                    'start_date': item['from'],
                    'end_date': item['to'],
                    'variable': 'education_country',
                    'value': item['country']
                },              {
                    'uuid': profile_data['uuid'],
                    'name': profile_data['name'],
                    'parent_uuid': profile_data['parent_uuid'],
                    'parent_name': profile_data['companieshouse_company_name'],
                    'group_id': None, 
                    'start_date': item['from'],
                    'end_date': item['to'],
                    'variable': 'education_city',
                    'value': item['city']
                },
                {
                    'uuid': profile_data['uuid'],
                    'name': profile_data['name'],
                    'parent_uuid': profile_data['parent_uuid'],
                    'parent_name': profile_data['companieshouse_company_name'],
                    'group_id': None, 
                    'start_date': item['from'],
                    'end_date': item['to'],
                    'variable': 'education_degree_description',
                    'value': item['description']
                }])
    return pd.DataFrame(flattened_records)
                
# Apply the function to the provided data
df_long_format = flatten_linkedin(linkedin_sample)
df_long_format

,uuid,name,parent_uuid,parent_name,group_id,start_date,end_date,variable,value
0,c4a15ea3-b525-5da9-a6a8-03beb1fe3fc2,Djordje Djokovic,a7d2f427-66ba-476a-81cc-171a5d806b22,Isize Limited,None,2013-12-01,2021-10-01,experience_organization,Maven Securities
1,c4a15ea3-b525-5da9-a6a8-03beb1fe3fc2,Djordje Djokovic,a7d2f427-66ba-476a-81cc-171a5d806b22,Isize Limited,None,2013-12-01,2021-10-01,experience_position,Trader
2,c4a15ea3-b525-5da9-a6a8-03beb1fe3fc2,Djordje Djokovic,a7d2f427-66ba-476a-81cc-171a5d806b22,Isize Limited,None,2013-12-01,2021-10-01,experience_country,GB
3,c4a15ea3-b525-5da9-a6a8-03beb1fe3fc2,Djordje Djokovic,a7d2f427-66ba-476a-81cc-171a5d806b22,Isize Limited,None,2013-12-01,2021-10-01,experience_city,London
4,c4a15ea3-b525-5da9-a6a8-03beb1fe3fc2,Djordje Djokovic,a7d2f427-66ba-476a-81cc-171a5d806b22,Isize Limited,None,2013-12-01,2021-10-01,experience_position_description,Heading Automated Options Market Making
5,c4a15ea3-b525-5da9-a6a8-03beb1fe3fc2,Djordje Djokovic,a7d2f427-66ba-476a-81cc-171a5d806b22,Isize Limited,None,2016-09-01,2021-03-01,experience_organization,iSIZE Technologies - Enhancing Video Streaming...
6,c4a15ea3-b525-5da9-a6a8-03beb1fe3fc2,Djordje Djokovic,a7d2f427-66ba-476a-81cc-171a5d806b22,Isize Limited,None,2016-09-01,2021-03-01,experience_position,Co-Founder and Shareholder
7,c4a15ea3-b525-5da9-a6a8-03beb1fe3fc2,Djordje Djokovic,a7d2f427-66ba-476a-81cc-171a5d806b22,Isize Limited,None,2016-09-01,2021-03-01,experience_country,GB
8,c4a15ea3-b525-5da9-a6a8-03beb1fe3fc2,Djordje Djokovic,a7d2f427-66ba-476a-81cc-171a5d806b22,Isize Limited,None,2016-09-01,2021-03-01,experience_city,London
9,c4a15ea3-b525-5da9-a6a8-03beb1fe3fc2,Djordje Djokovic,a7d2f427-66ba-476a-81cc-171a5d806b22,Isize Limited,None,2016-09-01,2021-03-01,experience_position_description,


In [77]:
ujson.dumps(linkedin_sample)

'{"parsing_date":"2023-08-22","uuid":"c4a15ea3-b525-5da9-a6a8-03beb1fe3fc2","parent_uuid":"a7d2f427-66ba-476a-81cc-171a5d806b22","companieshouse_id":"10388803","name":"Djordje Djokovic","full_name":"Djordje Djokovic","linkedin_name":"Djordje Djokovic, PhD","companieshouse_company_name":"Isize Limited","occupations":["Founder","Shareholder"],"linkedin_url":"https:\\/\\/www.linkedin.com\\/in\\/djordje-djokovic-phd-683589","country":"GB","city":"","experience":[{"items":[{"position":"Trader","from":"2013-12-01","to":"2021-10-01","days":2861,"country":"GB","city":"London","description":"Heading Automated Options Market Making"}],"organization":"Maven Securities","from":"2013-12-01","to":"2021-10-01","days":2861},{"items":[{"position":"Co-Founder and Shareholder","from":"2016-09-01","to":"2021-03-01","days":1642,"country":"GB","city":"London","description":""}],"organization":"iSIZE Technologies - Enhancing Video Streaming With Advanced Machine Learning","from":"2016-09-01","to":"2021-03-01

# todo: repopulate dissolved on!
# todo: test unique names for 100 records!
# todo: repopulate unique names!


## write all data based on list of names

### get uuids by name

In [28]:
names = ['elliptic', 'isize']
df_uuids = pd.DataFrame(get_uuids_from_crunchbase_organizations(names))
uuids = list(df_uuids['uuid'])
df_uuids[['uuid', 'name', 'founded_on']]

,uuid,name,founded_on
0,a7d2f427-66ba-476a-81cc-171a5d806b22,iSIZE Technologies,2016-09-22
1,5fd4dcb9-77ec-8903-5300-dd4c3da76670,Elliptic,2013-10-01
2,ab635b6c-1424-4434-b504-b741a65781ae,Mobilisize,2013-01-01
3,36e2cbf9-0004-4a43-a8d1-0964997f926b,Elliptic Marketing,2014-01-01
4,506e9b63-21ee-4878-96bc-a87abeed4484,Elliptic Ventures,2018-01-01
5,d07e06c9-9b49-48b5-b6fa-a24e20ef4160,Elliptical Machines,2019-01-01
6,46678256-73c8-4e43-9932-29c0fe0cf6a4,Elliptical Mobile Solutions,2018-01-01
7,b7e97401-102b-e71f-81b2-7ee8022a8cb9,EllipticalHub,2013-10-02
8,edb3c0af-d871-4ef1-fa32-87fab0a170d7,Verisize,2012-01-01


In [5]:
pd.DataFrame(get_data_from_pending(source=DataSource.linkedin.name, uuids_parent=uuids, force=True))

""


In [26]:
import json

# Define the class
class CompanyData:
    def __init__(self):
        self.data_storage = {}

    def add_record(self, tablename, id, company_id, entity_type, entity_id, variable, category, start_date, end_date, value):
        if tablename not in self.data_storage:
            self.data_storage[tablename] = []
        
        # Create the record
        record = {
            'id': id,
            'company_id': company_id,
            'entity_type': entity_type,
            'entity_id': entity_id,
            'variable': variable,
            'category': category,
            'start_date': start_date,
            'end_date': end_date,
            'value': value
        }
        # Store the record under the appropriate table name
        self.data_storage[tablename].append(record)

    def get_data_by_table(self, tablename):
        return self.data_storage.get(tablename, [])

    def get_all_data(self):
        # This method returns all data from all tables
        all_data = []
        for records in self.data_storage.values():
            all_data.extend(records)
        return all_data

company_data = CompanyData()

# Add shareholding records
company_data.add_record('shareholding', None, 1, 'Individual', 101, 'shareholding', 'Founder', '2022-01-01', '2023-01-01', 50)
company_data.add_record('shareholding', None, 1, 'Individual', 101, 'shareholding', 'Founder', '2023-01-01', '9999-12-31', 45)
company_data.add_record('shareholding', None, 1, 'Individual', 102, 'shareholding', 'Director', '2022-01-01', '2023-01-01', 30)
company_data.add_record('shareholding', None, 1, 'Individual', 102, 'shareholding', 'Director', '2023-01-01', '9999-12-31', 35)

# Add investment records
company_data.add_record('investment', None, 1, 'Investor', 201, 'investment', 'Series A', '2022-06-01', '9999-12-31', 1000000)
company_data.add_record('investment', None, 1, 'Investor', 202, 'investment', 'Series B', '2023-06-01', '9999-12-31', 2000000)

# Add patent records
company_data.add_record('patent', None, 1, 'Company', 301, 'patent', 'Claims', '2022-05-01', '9999-12-31', 10)
company_data.add_record('patent', None, 1, 'Company', 301, 'patent', 'Citations', '2022-05-01', '9999-12-31', 5)
company_data.add_record('patent', None, 1, 'Company', 302, 'patent', 'Claims', '2022-07-01', '9999-12-31', 15)
company_data.add_record('patent', None, 1, 'Company', 302, 'patent', 'Citations', '2022-07-01', '9999-12-31', 2)

# Add directorship record
company_data.add_record('directorship', None, 1, 'Individual', 102, 'officers', 'Director', '2022-01-01', '2023-01-01', 1)

# Add education records
company_data.add_record('education', None, 1, 'Individual', 101, 'education', 'Level', '2022-01-01', '9999-12-31', 'PhD')
company_data.add_record('education', None, 1, 'Individual', 101, 'education', 'Field', '2022-01-01', '9999-12-31', 'STEM')
company_data.add_record('education', None, 1, 'Individual', 102, 'education', 'Level', '2022-01-01', '9999-12-31', "Master's")
company_data.add_record('education', None, 1, 'Individual', 102, 'education', 'Field', '2022-01-01', '9999-12-31', 'Business')

# Add individual patent records
company_data.add_record('patent', None, 1, 'Individual', 101, 'patent', 'Claims', '2022-09-01', '9999-12-31', 3)
company_data.add_record('patent', None, 1, 'Individual', 101, 'patent', 'Citations', '2022-09-01', '9999-12-31', 7)
company_data.add_record('patent', None, 1, 'Individual', 102, 'patent', 'Claims', '2022-11-01', '9999-12-31', 5)
company_data.add_record('patent', None, 1, 'Individual', 102, 'patent', 'Citations', '2022-11-01', '9999-12-31', 2)

# Assuming you want to print or access these records, you might use:
# Print all shareholding records
print(company_data.get_data_by_table('shareholding'))

# Print all records from all categories
print(json.dumps(company_data.get_all_data(), indent=4))


[{'id': None, 'company_id': 1, 'entity_type': 'Individual', 'entity_id': 101, 'variable': 'shareholding', 'category': 'Founder', 'start_date': '2022-01-01', 'end_date': '2023-01-01', 'value': 50}, {'id': None, 'company_id': 1, 'entity_type': 'Individual', 'entity_id': 101, 'variable': 'shareholding', 'category': 'Founder', 'start_date': '2023-01-01', 'end_date': '9999-12-31', 'value': 45}, {'id': None, 'company_id': 1, 'entity_type': 'Individual', 'entity_id': 102, 'variable': 'shareholding', 'category': 'Director', 'start_date': '2022-01-01', 'end_date': '2023-01-01', 'value': 30}, {'id': None, 'company_id': 1, 'entity_type': 'Individual', 'entity_id': 102, 'variable': 'shareholding', 'category': 'Director', 'start_date': '2023-01-01', 'end_date': '9999-12-31', 'value': 35}]
[
    {
        "id": null,
        "company_id": 1,
        "entity_type": "Individual",
        "entity_id": 101,
        "variable": "shareholding",
        "category": "Founder",
        "start_date": "2022-01

In [15]:
shareholdings = [
    {"id": 1, "company_id": 1, "entity_type": "Individual", "entity_id": 101, "variable": "shareholding", "category": "Founder", "start_date": "2022-01-01", "end_date": "2023-01-01", "value": 50},
    {"id": 2, "company_id": 1, "entity_type": "Individual", "entity_id": 101, "variable": "shareholding", "category": "Founder", "start_date": "2023-01-01", "end_date": "9999-12-31", "value": 45},
    {"id": 3, "company_id": 1, "entity_type": "Individual", "entity_id": 102, "variable": "shareholding", "category": "Director", "start_date": "2022-01-01", "end_date": "2023-01-01", "value": 30},
    {"id": 4, "company_id": 1, "entity_type": "Individual", "entity_id": 102, "variable": "shareholding", "category": "Director", "start_date": "2023-01-01", "end_date": "9999-12-31", "value": 35},
]

investments = [
    {"id": 5, "company_id": 1, "entity_type": "Investor", "entity_id": 201, "variable": "investment", "category": "Series A", "start_date": "2022-06-01", "end_date": "2022-06-01", "value": 1000000},
    {"id": 6, "company_id": 1, "entity_type": "Investor", "entity_id": 202, "variable": "investment", "category": "Series B", "start_date": "2023-06-01", "end_date": "2023-06-01", "value": 2000000},
]

patents = [
    {"id": 7, "company_id": 1, "entity_type": "Company", "entity_id": 1, "variable": "patent", "category": "claims", "start_date": "2022-05-01", "end_date": "2022-05-01", "value": 10},
    {"id": 8, "company_id": 1, "entity_type": "Company", "entity_id": 1, "variable": "patent", "category": "citations", "start_date": "2022-05-01", "end_date": "2022-05-01", "value": 5},
    {"id": 9, "company_id": 1, "entity_type": "Company", "entity_id": 1, "variable": "patent", "category": "claims", "start_date": "2022-07-01", "end_date": "2022-07-01", "value": 15},
    {"id": 10, "company_id": 1, "entity_type": "Company", "entity_id": 1, "variable": "patent", "category": "citations", "start_date": "2022-07-01", "end_date": "2022-07-01", "value": 2},
]

education = [
    {"id": 11, "company_id": 1, "entity_type": "Individual", "entity_id": 101, "variable": "education", "category": "PhD", "start_date": "2022-01-01", "end_date": "9999-12-31", "value": "STEM"},
    {"id": 12, "company_id": 1, "entity_type": "Individual", "entity_id": 102, "variable": "education", "category": "Master's", "start_date": "2022-01-01", "end_date": "9999-12-31", "value": "Business"},
]

directorships = [
    {"id": 13, "company_id": 1, "entity_type": "Individual", "entity_id": 102, "variable": "officers", "category": "Director", "start_date": "2022-01-01", "end_date": "2023-01-01", "value": 1},
]


In [16]:
def transform_to_long_format(tables):
    long_format_frames = []
    
    for table in tables:
        df = pd.DataFrame(table)
        long_format_frames.append(df)
    
    result = pd.concat(long_format_frames, ignore_index=True)
    return result


In [8]:
# Put all the tables into a list
tables = [shareholdings, investments, patents, education, directorships]

# Call the function
long_format_df = transform_to_long_format(tables)

# Now 'long_format_df' is your dataframe in long format
print(long_format_df)

NameError: name 'shareholdings' is not defined

JSONDecodeError: Invalid control character at: line 1 column 53680 (char 53679)